**В этом ноутбуке показаны попытки 4-5 создания рек.системы**  
Процесс создания признаков для трен/тест датасетов показан в   
главном ноутбуке с лучшей попыткой 6.  
Ниже краткое описание различий каждой попытки

**Ver_4**  
- 150 кандидатов на сессию
- фичи : \
Aid \
'item_quantity_in_df' Кол-во aid в датасете.   
'sessions_w_item' Сколько session с этим aid. \
'mean_item_type' Средний тип этого aid. \
'rank_pop' Популярность aid внутри типа \
Session \
'session_quantity_in_df' Кол-во этой session в датасете \
'items_in_session' Сколько aid в этой session \
'mean_session_type' Какой средний тип aid в этой session \
Session x Aid    
'rank' Ранг aid в сессии по ковиз-матрице  
- 1/3 train data
- Undersample : 0.012 ratio - clicks, 0.05 ratio - carts, 0.02 ratio -  orders


**Ver_4.1**  
- 50 кандидатов на сессию
- фичи : как в ver_4  
- 1/3 train data
- Undersample : 0.43 ratio - clicks, 0.0624 ratio - carts, 0.0624 ratio - orders

**Ver_5**  
- 50 кандидатов на сессию
- фичи : \
Aid \
'item_quantity_in_df' Кол-во aid в датасете.   
'sessions_w_item' Сколько session с этим aid. \
'mean_item_type' Средний тип этого aid. \
'rank_pop' Популярность aid внутри типа \
Session \
'session_quantity_in_df' Кол-во этой session в датасете \
'items_in_session' Сколько aid в этой session \
'mean_session_type' Какой средний тип aid в этой session \
'session_len' Длина сессии (в минутах)  
'last_aid_type' Последнее действие каждого типа в session  
Session x Aid    
'rank' Ранг aid в сессии по ковиз-матрице   
'cos_sim' Косинусная схожесть session-aid WORD2WEC   
- 1/3 train data
- Undersample : 0.43 ratio - clicks, 0.0624 ratio - carts, 0.0624 ratio - orders


In [ ]:
# main_path = '/Users/stanislavkrupnov/Jup.Notebook'
main_path = '/content/drive/Othercomputers/Mac/Jup.Notebook'

# Функции

In [ ]:
def metric_eval(predictions: pd.DataFrame, valid: pd.DataFrame,
                id2type: dict) -> tuple:
    """
    Вычисляет метрики для оценки качества модели.

    Parameters:
    - predictions (pd.DataFrame): DataFrame с предсказаниями модели.
    - valid (pd.DataFrame): DataFrame с данными для валидации.
    - id2type (dict): Словарь для преобразования индексов типов в соответствующие строки.

    Returns:
    - tuple: Кортеж, содержащий локальную метрику и реколл для каждого типа.
    """
    # Преобразование индексов типов в соответствующие строки
    valid.type = valid.type.map(lambda idx: id2type[idx])

    # Создание списка с метками для каждой сессии и типа
    ground_truth = valid.groupby(['session', 'type'])['aid'].apply(list)
    ground_truth = ground_truth.reset_index().rename(columns={'aid': 'labels'})

    # Обрезка меток для типа 'clicks' до одной метки
    ground_truth.loc[ground_truth.type == 'clicks',
                     'labels'] = ground_truth.loc[ground_truth.type ==
                                                  'clicks', 'labels'].str[:1]

    # Объединение предсказаний с истинными метками
    submission_with_gt = predictions.merge(
        ground_truth[['session', 'type', 'labels']],
        how='left',
        on=['session', 'type'])

    # Отбрасывание сессий без истинных меток
    submission_with_gt = submission_with_gt[~submission_with_gt.labels_y.isna(
    )]

    # Вычисление количества совпадений между предсказанными и истинными метками
    submission_with_gt['hits'] = submission_with_gt.apply(
        lambda df: len(set(df.labels_x).intersection(set(df.labels_y))),
        axis=1)

    # Вычисление количества истинных меток для каждого типа
    submission_with_gt['gt_count'] = submission_with_gt.labels_y.str.len(
    ).clip(0, 20)

    # Вычисление реколла для каждого типа
    recall_per_type = submission_with_gt.groupby([
        'type'
    ])['hits'].sum() / submission_with_gt.groupby(['type'])['gt_count'].sum()

    # Вычисление локальной метрики на основе реколла для каждого типа
    local_validation_score = (recall_per_type * pd.Series({
        'clicks': 0.10,
        'carts': 0.30,
        'orders': 0.60
    })).sum()

    return local_validation_score, recall_per_type

In [ ]:
def read_covis_to_dict(cov_ver: str, cov_type: str, n: int) -> dict:
    '''
    Читает ковиз-матрицу из нескольких файлов и возвращает словарь.

    Parameters:
    - cov_ver (str): Версия ковиз-матрицы.
    - cov_type (str): Тип ковиз-матрицы (b_tr или lv).
    - n (int): Количество файлов с матрицей.

    Returns:
    - dict: Словарь, представляющий ковиз-матрицу.
    '''
    for k in tqdm(range(0, n)):
        path = f'{main_path}/c/{cov_type}_{cov_ver}/{cov_type}_covis_matrix_{k}.parquet'

        if k == 0:
            covis_matrix = pd.read_parquet(path)
            covis_matrix = covis_matrix.groupby('aid_x').aid_y.apply(
                list).to_dict()
        else:
            df = pd.read_parquet(path)
            df = df.groupby('aid_x').aid_y.apply(list).to_dict()
            covis_matrix.update(df)

    return covis_matrix

In [ ]:
def read_parquet_in_chunks(file_path: str, batch_num: int):
    '''
    Читает Parquet-файл порциями и возвращает DataFrame.

    Parameters:
    - file_path (str): Путь к Parquet-файлу.
    - batch_num (int): Количество порций для чтения.

    Yields:
    - pd.DataFrame: DataFrame, представляющий порцию данных из Parquet-файла.
    '''
    table = pq.read_table(file_path)
    num_rows = len(table)
    batch_size = num_rows // batch_num

    for i in range(0, num_rows, batch_size):
        yield table[i:i + batch_size].to_pandas()

In [ ]:
def create_labels(n: int,
                  inp_dict: dict,
                  covis: dict,
                  top_pop_aids: set,
                  cov_f: bool,
                  b: int = -1) -> dict:
    '''
    Создает метки для сессий на основе матрицы ко-посещений.

    Parameters:
    - n (int): Количество кандидатов на сессию.
    - inp_dict (dict): Словарь с записями сессий.
    - covis (dict): Матрица ко-посещений.
    - top_pop_aids (set): Топ популярных товаров.
    - cov_f (bool): Создавать ли признак из ковиз-матрицы.
    - b (int): Номер файла для сохранения, если cov_f=True.

    Returns:
    - dict: Словарь с кандидатами.
    '''
    labels = {}
    featuress = {}

    for session, aids in tqdm(inp_dict.items()):
        # Инициализация словаря для подсчета aids
        aid_counter = Counter()

        # Цикл по всем aids в сессии
        for aid in aids:
            # Если aid есть в covis_matrix, увеличиваем его счетчик
            if aid in covis:
                aid_counter.update(covis[aid])

        if len(set(aids)) >= n:  # Оставлю только последние n уникальные aids
            labels[session] = list(dict.fromkeys(aids))[-n:]
        else:
            # Получаем топ встречающихся aids
            top_aids = [
                aid
                for aid, count in aid_counter.most_common(n - len(set(aids)))
            ]

            # Добавляем исходные aids и топ встречающихся aids в labels
            labels[session] = list(set(list(set(aids)) + list(set(top_aids))))

            if len(set(aids)) < n:
                pops = list(top_pop_aids.difference(set(labels[session])))
                labels[session] += pops[:n - len(labels[session])]

        if cov_f == False:
            continue
        else:
            # Создание словаря с признаком - рангом (по ко-виз матрице) каждого кандидата внутри сессии
            for aid in labels[session]:
                if aid in list(aid_counter.keys()):
                    if session not in featuress:
                        featuress[session] = {}
                    featuress[session][aid] = aid_counter[aid]

    if cov_f == True:
        if b == -1:  # Если мы в функции create_labels
            path_ = f'{main_path}/tr/{ver_folder}/featuress.pickle'
        else:  # Если мы в функции create_labels_in_batches
            path_ = f'{main_path}/tr/{ver_folder}/featuress_{b}.pickle'

        with open(path_, 'wb') as file:
            pickle.dump(featuress, file)

        print(
            f'Словарь с признаком-рангом по ковиз-матрице сохранен в : {path_}'
        )

    return labels

In [ ]:
def create_labels_in_batches(k: int, n: int, inp_dict: dict, covis: dict,
                             top_pop_aids: set, cov_f: bool) -> dict:
    '''
    Создает метки для сессий на основе матрицы ко-посещений по батчам.

    Parameters:
    - k (int): Количество батчей.
    - n (int): Количество кандидатов на сессию.
    - inp_dict (dict): Словарь с записями сессий.
    - covis (dict): Матрица ко-посещений.
    - top_pop_aids (set): Топ популярных товаров.
    - cov_f (bool): Создавать ли признак из ковиз-матрицы.

    Returns:
    - dict: Словарь с кандидатами.
    '''
    labels = {}
    partt = len(inp_dict) // k

    for batch in tqdm(range(k), desc='part'):

        start = partt * batch
        end = partt * (batch + 1)
        slicee_ = list(inp_dict.items())[start:end]
        slicee = dict(slicee_)

        labels_ = create_labels(n=n,
                                inp_dict=slicee,
                                covis=covis,
                                top_pop_aids=top_pop_aids,
                                cov_f=cov_f,
                                b=batch)
        labels.update(labels_)

    return labels

In [ ]:
def create_pop_features(for_features_df: pd.DataFrame) -> pd.DataFrame:
    '''
    Создает признак ранжирования aid по встречаемости в группах по типам.

    Parameters:
    - for_features_df (pd.DataFrame): Датасет с данными для признака.

    Returns:
    - pd.DataFrame: Датасет с признаком.
    '''
    # Создадим пустой DataFrame, в который будем записывать результаты
    result_df = pd.DataFrame()

    # Сгруппируем исходный DataFrame по столбцу 'type'
    grouped = for_features_df.groupby('type')

    id_2_type = {0: 'clicks', 1: 'carts', 2: 'orders'}

    # Пройдемся по каждой группе и выполним ранжирование aid по встречаемости
    for name, group in tqdm(grouped):
        # Считаем количество вхождений каждого aid в текущей группе и создадим временный DataFrame
        aid_counts = group['aid'].value_counts().reset_index()
        aid_counts.columns = ['aid', 'count']

        # Добавим столбец с рангом, используя функцию rank()
        aid_counts['rank'] = aid_counts['count'].rank(ascending=False,
                                                      method='dense')
        aid_counts.drop(columns=['count'], inplace=True)

        # Присоединим результат к исходной группе и добавим в общий результатный DataFrame
        merged_group = pd.merge(group, aid_counts, on='aid', how='left')

        # Создадим имена для столбцов рангов на основе типа 'type'
        rank_column_name = 'rank_' + id_2_type[name]
        merged_group.rename(columns={'rank': rank_column_name}, inplace=True)

        result_df = pd.concat([result_df, merged_group], ignore_index=True)

        result_df.fillna(-1, inplace=True)
        result_df.rank_clicks = result_df.rank_clicks.astype(np.int32)
        result_df.rank_carts = result_df.rank_carts.astype(np.int32)
        result_df.rank_orders = result_df.rank_orders.astype(np.int32)
        result_df.reset_index(inplace=True, drop=True)

    return result_df

In [ ]:
def add_targets(dataframe_: pd.DataFrame, targets: pd.DataFrame,
                targ_type: int) -> pd.DataFrame:
    '''
    Добавляет таргеты к датафрейму.

    Parameters:
    - dataframe_ (pd.DataFrame): Датафрейм, которому нужны таргеты.
    - targets (pd.DataFrame): Датафрейм с таргетами всех типов.
    - targ_type (int): Текущий тип таргета.

    Returns:
    - pd.DataFrame: Датафрейм с нужными таргетами.
    '''
    targets_ = targets.loc[targets['type'] == targ_type,
                           ['session', 'aid', 'type']].rename(columns={
                               'type': 'target'
                           }).copy()
    targets_['target'] = 1
    targets_ = targets_.drop_duplicates()
    merge_cols = ["session", "aid"]
    dataframe = dataframe_.merge(targets_, on=merge_cols, how="left")
    dataframe = dataframe.fillna(0)
    dataframe.target = dataframe.target.astype(np.uint8)

    return dataframe

In [ ]:
def w_pbar(pbar: tqdm, func: callable):
    '''
    Оборачивает функцию, обновляя tqdm прогресс бар.

    Parameters:
    - pbar (tqdm): Прогресс бар.
    - func (callable): Функция.

    Returns:
    - callable: Обернутая функция.
    '''
    def foo(*args, **kwargs):
        pbar.update(1)
        return func(*args, **kwargs)

    return foo

# import

In [ ]:
import os
import sys
from tqdm import tqdm
import glob
import pyarrow.parquet as pq
import pickle
import pandas as pd
import gc
import numpy as np
import itertools
from collections import Counter
import random
from itertools import product
import xgboost as xgb
from sklearn.model_selection import GroupKFold
import gdown
from google.colab import drive
drive.mount('/content/drive')
import polars as pl
import shutil
from datetime import datetime as dt
from collections import defaultdict


Mounted at /content/drive


# Ver_4

- b_tr/lv_covis_matrix base
- ver_4 xgb
- 150 кандидатов
- фичи :
(база из 6 основных) + популярности товара по типам (1 признак ), counts из covis(featuress dict)
- 1/3 data

In [ ]:
ver = '4'
ver_folder = f'ver_{ver}'
cand_n = 150

## train_candidates

### создание общего списка tr_candidates

In [ ]:
tr_candidates_sessions = pd.read_parquet(f'{main_path}/s/tr_candidates_sessions.parquet')
tr_2_first_w = pd.read_parquet(f'{main_path}/s/tr_2_first_w.parquet')

In [ ]:
covis_matrix = read_covis_to_dict(cov_ver = 'base',
                                  cov_type = 'b_tr',
                                  n = 20)

100%|███████████████████████████████████████████| 20/20 [00:15<00:00,  1.28it/s]


In [ ]:
top_pop_aids = pd.concat([tr_candidates_sessions, tr_2_first_w])
top_pop_aids = set(top_pop_aids['aid'].value_counts().index[:cand_n].tolist())

In [ ]:
%%time
# Преобразование датафрейма в словарь
tgl_dict = tr_candidates_sessions.sort_values(["session", "ts"]).groupby('session')['aid'].apply(list).to_dict()

CPU times: user 30.6 s, sys: 1.96 s, total: 32.6 s
Wall time: 32.8 s


In [ ]:
del tr_candidates_sessions, tr_2_first_w

In [ ]:
len(tgl_dict)//3

1547581

In [ ]:
# Если нужен слайс
slicee_ = list(tgl_dict.items())[:1547581] # 1547581 - одна третья данных
slicee = dict(slicee_)
tgl_dict = slicee

In [ ]:
labels = create_labels(n = cand_n,
                      inp_dict = tgl_dict,
                      covis = covis_matrix,
                      top_pop_aids = top_pop_aids,
                      cov_f = True)

100%|███████████████████████████████| 1547581/1547581 [05:47<00:00, 4450.41it/s]


Словарь с признаком-рангом по ковиз-матрице сохранен в : /Users/stanislavkrupnov/Jupyter_notebook/Pet_project/tr/ver_4/featuress.pickle


In [ ]:
sessions = []
aid_lists = []

for session, aids in tqdm(labels.items()):
    sessions.append(session)
    aid_lists.append(aids)

100%|████████████████████████████| 1547581/1547581 [00:00<00:00, 5877369.07it/s]


In [ ]:
tr_candidates_ = pd.DataFrame({'session': sessions, 'labels': aid_lists})

In [ ]:
tr_candidates_['session'] = tr_candidates_['session'].astype(np.int32)

In [ ]:
folder_path = f'{main_path}/tr/{ver_folder}/tr_candidates'
os.makedirs(folder_path, exist_ok=True)

partt = len(tr_candidates_)//5

for batch in tqdm(range(5), desc = 'part'):

    start = partt * batch
    end = partt + start
    print(start, end)

    if batch == 4:
        batch_ = tr_candidates_.loc[start:end]
    else:
        batch_ = tr_candidates_.loc[start:end-1]

    batch_ = batch_.explode('labels')

    batch_['labels'] = batch_['labels'].astype(np.int32)

    print(batch_[batch_['session'] == 21404]['labels'].count())

    batch_.to_parquet(f'{folder_path}/tr_candidates_{batch}.parquet')


part:   0%|                                               | 0/5 [00:00<?, ?it/s]

0 309516
150


part:  20%|███████▊                               | 1/5 [00:07<00:31,  7.92s/it]

309516 619032
0


part:  40%|███████████████▌                       | 2/5 [00:15<00:22,  7.64s/it]

...

part:  60%|███████████████████████▍               | 3/5 [00:22<00:14,  7.44s/it]

928548 1238064
0


part:  80%|███████████████████████████████▏       | 4/5 [00:29<00:07,  7.30s/it]

1238064 1547580
0


part: 100%|███████████████████████████████████████| 5/5 [00:36<00:00,  7.28s/it]


### Преобразование covis_features

In [ ]:
with open(f'{main_path}/tr/{ver_folder}/featuress.pickle', 'rb') as file:

    featuress = pickle.load(file)

features_df = (pd.DataFrame.from_dict([(session_id, aid, rank)
                                for session_id, items in featuress.items()
                                for aid, rank in items.items()]))

In [ ]:
features_df.columns = ['session','aid','rank']
features_df['session'] = features_df['session'].astype(np.int32)
features_df['aid'] = features_df['aid'].astype(np.int32)
features_df['rank'] = features_df['rank'].astype(np.uint8)

In [ ]:
folder_path = f'{main_path}/tr/{ver_folder}/covis_features'
os.makedirs(folder_path, exist_ok=True)

features_df.reset_index(drop=True, inplace=True)

partt = len(features_df)//5

for batch in tqdm(range(5), desc = 'part'):

    start = partt * batch
    end = partt * (batch + 1)
    batch_ = features_df.loc[start:end]

    batch_.to_parquet(f'{main_path}/tr/{ver_folder}/covis_features/covis_features_{batch}.parquet')


part: 100%|███████████████████████████████████████| 5/5 [00:03<00:00,  1.37it/s]


### Создание признаков  популярности товара по типам

In [ ]:
tr_candidates_sessions = pd.read_parquet(f'{main_path}/s/tr_candidates_sessions.parquet')
tr_2_first_w = pd.read_parquet(f'{main_path}/s/tr_2_first_w.parquet')
feature_name = 'pop_features'

In [ ]:
for_features_df = pd.concat([tr_candidates_sessions, tr_2_first_w])
for_features_df = for_features_df.drop(columns = ['ts'])

In [ ]:
# Сгруппируем исходный DataFrame по столбцу 'type'
grouped = for_features_df.groupby('type')

id_2_type = {0:'clicks', 1: 'carts', 2:'orders'}

# Пройдемся по каждой группе и выполним ранжирование aid по встречаемости
for name, group in tqdm(grouped):
    # Считаем количество вхождений каждого aid в текущей группе и создадим временный DataFrame
    aid_counts = group['aid'].value_counts().reset_index()
    aid_counts.columns = ['aid', 'count']

    # Добавим столбец с рангом, используя функцию rank()
    aid_counts['rank_pops'] = aid_counts['count'].rank(ascending=False, method='dense')
    aid_counts.drop(columns = ['count'], inplace = True)

    # Присоединим результат к исходной группе и добавим в общий результатный DataFrame
    group = group.drop(columns = ['session', 'type']).drop_duplicates(subset=['aid'])
    merged_group = pd.merge(group, aid_counts, on='aid', how='left')
    merged_group.fillna(-1, inplace = True)
    merged_group['rank_pops'] = merged_group['rank_pops'].astype(np.int32)
    merged_group.reset_index(inplace = True, drop = True)

    merged_group.to_parquet(f'{main_path}/tr/{ver_folder}/pop_features_{id_2_type[name]}.parquet')

100%|█████████████████████████████████████████████| 3/3 [00:05<00:00,  1.68s/it]


### Добавление признаков, таргетов к tr_candidates

In [ ]:
tr_item_features = pd.read_parquet(f'{main_path}/tr/{ver_folder}/tr_item_features.parquet')
tr_user_features = pd.read_parquet(f'{main_path}/tr/{ver_folder}/tr_user_features.parquet')

In [ ]:
targets = pd.read_parquet(f'{main_path}/s/targets.parquet')

**Добавление признаков + таргетов**

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_folder}/tr_candidates/*')
file_list_covis_f = glob.glob(f'{main_path}/tr/{ver_folder}/covis_features/*')
file_list_pops = glob.glob(f'{main_path}/tr/{ver_folder}/pop_features_*.parquet')

file_list.sort()
file_list_covis_f.sort()
file_list_pops.sort()

# Цикл для чтения и объединения файлов
for iter_numb, (file, file_2) in tqdm(enumerate(zip(file_list, file_list_covis_f)), desc = f'parts'):

    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)
    covis_f = pd.read_parquet(file_2)

    df_ = df_.rename(columns = {'labels': 'aid'})
    df_ = df_.merge(tr_item_features, on = 'aid', how='left').fillna(-1)
    df_ = df_.merge(tr_user_features, on = 'session', how='left').fillna(-1)
    print('Done 1/3')
    df_ = df_.merge(covis_f, on = ['session', 'aid'], how='left').fillna(-1)
    print('Done 2/3')


    for (type_, file_path_, index) in tqdm((zip(['carts', 'clicks', 'orders'], file_list_pops, [1,0,2])), desc = f'{iter_numb}_types'):

        folder_path = f'{main_path}/tr/{ver_folder}/tr_candidates_{type_}'
        os.makedirs(folder_path, exist_ok=True)

        # Добавление признаков популярности
        columns_ = ['aid', 'rank_pops']
        pop_features = pd.read_parquet(file_path_)
        df_m = df_.merge(pop_features[columns_], on = 'aid', how='left').fillna(-1)
        print('Done 3/3')

        df_m = add_targets(dataframe_ = df_m,
                           targets = targets,
                           targ_type = index)

        df_m['rank_pops'] = df_m['rank_pops'].astype(np.int32)
        df_m['rank'] = df_m['rank'].astype(np.int32)

        print('Done 4/3')
#         df_.to_parquet(f"{main_path}/tr/{ver_folder}/tr_candidates_{type_}/tr_candidates_{type_}_{iter_numb}.parquet")
        df_m.to_parquet(f"{main_path}/tr/{ver_folder}/tr_candidates_{type_}/tr_candidates_{type_}_{iter_numb}.parquet")

parts: 0it [00:00, ?it/s]

Done 1/3
Done 2/3



0_types: 0it [00:00, ?it/s]

Done 3/3
Done 4/3



0_types: 1it [00:24, 24.45s/it]

...


4_types: 1it [00:26, 26.79s/it]

Done 3/3
Done 4/3



4_types: 2it [00:48, 23.67s/it]

Done 3/3
Done 4/3



4_types: 3it [01:10, 23.47s/it]
parts: 5it [07:06, 85.36s/it]


In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_folder}/tr_candidates_clicks/*')

# Создание пустого DataFrame для сбора данных
df = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    df = pd.concat([df, df_], ignore_index=True)

100%|█████████████████████████████████████████████| 5/5 [00:20<00:00,  4.16s/it]


In [ ]:
x = df.groupby('session')['aid'].count().reset_index()

In [ ]:
xx = df.groupby('session')['aid'].nunique().reset_index()

In [ ]:
a = 0
for session in tqdm(x['session']) :

    if (x.loc[x['session'] == session, 'aid'] != xx.loc[xx['session'] == session, 'aid']).any():
        print(session)
        a+=1

## test_candidates

### Создание списка test_candidates CPU ONLY

In [ ]:
ver = 4
ver_folder = f'ver_{ver}'
cand_n = 150

In [ ]:
test_candidates_sessions = pd.read_parquet(f'{main_path}/s/test_candidates_sessions.parquet')
tr_3_first_w = pd.read_parquet(f'{main_path}/s/tr_3_first_w.parquet')

In [ ]:
covis_matrix = read_covis_to_dict(cov_ver = 'base',
                                  cov_type = 'lv',
                                  n = 80)

100%|██████████| 80/80 [03:21<00:00,  2.52s/it]


In [ ]:
top_pop_aids = pd.concat([tr_3_first_w, test])
top_pop_aids = set(top_pop_aids['aid'].value_counts().index[:cand_n].tolist())

In [ ]:
# Преобразование датафрейма в словарь
test_dict = test_candidates_sessions.sort_values(["session", "ts"]).groupby('session')['aid'].apply(list).to_dict()

In [ ]:
sizee = len(test_dict) // 3

In [ ]:
# Если нужен слайс
slicee_ = list(test_dict.items())[:sizee] # sizee - одна третья данных
slicee = dict(slicee_)
test_dict = slicee

In [ ]:
labels = create_labels(n = cand_n,
                      inp_dict = test_dict,
                      covis = covis_matrix,
                      top_pop_aids = top_pop_aids,
                      cov_f = False)

100%|██████████| 600417/600417 [00:54<00:00, 11024.06it/s]


In [ ]:
sessions = []
aid_lists = []

for session, aids in tqdm(labels.items()):
    sessions.append(session)
    aid_lists.append(aids)

100%|██████████| 600417/600417 [00:00<00:00, 2323076.82it/s]


In [ ]:
test_candidates_ = pd.DataFrame({'session': sessions, 'labels': aid_lists})
test_candidates_['session'] = test_candidates_['session'].astype(np.int32)

In [ ]:
folder_path = f'{main_path}/lv/{ver_folder}/test_candidates'
os.makedirs(folder_path, exist_ok=True)

partt = len(test_candidates_)//5

for batch in tqdm(range(5), desc = 'part'):

    start = partt * batch
    end = partt + start
    print(start, end)

    if batch == 4:
        batch_ = test_candidates_.loc[start:end]
    else:
        batch_ = test_candidates_.loc[start:end-1]

    batch_ = batch_.explode('labels')

    batch_['labels'] = batch_['labels'].astype(np.int32)

    batch_.to_parquet(f'{folder_path}/test_candidates_{batch}.parquet')


part:   0%|          | 0/5 [00:00<?, ?it/s]

0 120083


part:  20%|██        | 1/5 [00:03<00:14,  3.60s/it]

120083 240166


part:  40%|████      | 2/5 [00:07<00:10,  3.65s/it]

...

part:  60%|██████    | 3/5 [00:10<00:07,  3.59s/it]

360249 480332


part:  80%|████████  | 4/5 [00:14<00:03,  3.62s/it]

480332 600415


part: 100%|██████████| 5/5 [00:18<00:00,  3.65s/it]


In [ ]:
del test_candidates_

### Преобразование covis - features

In [ ]:
with open(f'{main_path}/lv/{ver_folder}/featuress.pickle', 'rb') as file:

    featuress = pickle.load(file)

features_df = (pd.DataFrame.from_dict([(session_id, aid, rank)
                                for session_id, items in featuress.items()
                                for aid, rank in items.items()]))

In [ ]:
features_df.columns = ['session','aid','rank']
features_df['session'] = features_df['session'].astype(np.int32)
features_df['aid'] = features_df['aid'].astype(np.int32)
features_df['rank'] = features_df['rank'].astype(np.uint8)

In [ ]:
folder_path = f'{main_path}/lv/{ver_folder}/covis_features'
os.makedirs(folder_path, exist_ok=True)

features_df.reset_index(drop=True, inplace=True)

partt = len(features_df)//5

for batch in tqdm(range(5), desc = 'part'):

    start = partt * batch
    end = partt * (batch + 1)
    batch_ = features_df.loc[start:end]

    batch_.to_parquet(f'{main_path}/lv/{ver_folder}/covis_features/covis_features_{batch}.parquet')


part: 100%|██████████| 5/5 [00:02<00:00,  2.15it/s]


### Создание признаков  популярности товара по типам

In [ ]:
test_candidates_sessions = pd.read_parquet(f'{main_path}/s/test_candidates_sessions.parquet')
tr_3_first_w = pd.read_parquet(f'{main_path}/s/tr_3_first_w.parquet')
feature_name = 'pop_features'

In [ ]:
for_features_df = pd.concat([test_candidates_sessions, tr_3_first_w])
for_features_df = for_features_df.drop(columns = ['ts'])

In [ ]:
# Сгруппируем исходный DataFrame по столбцу 'type'
grouped = for_features_df.groupby('type')

id_2_type = {0:'clicks', 1: 'carts', 2:'orders'}

# Пройдемся по каждой группе и выполним ранжирование aid по встречаемости
for name, group in tqdm(grouped):
    # Считаем количество вхождений каждого aid в текущей группе и создадим временный DataFrame
    aid_counts = group['aid'].value_counts().reset_index()
    aid_counts.columns = ['aid', 'count']

    # Добавим столбец с рангом, используя функцию rank()
    aid_counts['rank_pops'] = aid_counts['count'].rank(ascending=False, method='dense')
    aid_counts.drop(columns = ['count'], inplace = True)

    # Присоединим результат к исходной группе и добавим в общий результатный DataFrame
    group = group.drop(columns = ['session', 'type']).drop_duplicates(subset=['aid'])
    merged_group = pd.merge(group, aid_counts, on='aid', how='left')
    merged_group.fillna(-1, inplace = True)
    merged_group['rank_pops'] = merged_group['rank_pops'].astype(np.int32)
    merged_group.reset_index(inplace = True, drop = True)

    merged_group.to_parquet(f'{main_path}/lv/{ver_folder}/pop_features_{id_2_type[name]}.parquet')

100%|██████████| 3/3 [00:13<00:00,  4.55s/it]


### Добавление признаков к test_candidates

In [ ]:
lv_item_features = pd.read_parquet(f'{main_path}/lv/{ver_folder}/lv_item_features.parquet')
lv_user_features = pd.read_parquet(f'{main_path}/lv/{ver_folder}/lv_user_features.parquet')

**Добавление признаков**

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/lv/{ver_folder}/test_candidates/*')
file_list_covis_f = glob.glob(f'{main_path}/lv/{ver_folder}/covis_features/*')
file_list_pops = glob.glob(f'{main_path}/lv/{ver_folder}/pop_features_*.parquet')

file_list.sort()
file_list_covis_f.sort()
file_list_pops.sort()

# Цикл для чтения и объединения файлов
for iter_numb, (file, file_2) in tqdm(enumerate(zip(file_list, file_list_covis_f)), desc = f'parts'):

    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)
    covis_f = pd.read_parquet(file_2)

    df_ = df_.rename(columns = {'labels': 'aid'})
    df_ = df_.merge(lv_item_features, on = 'aid', how='left').fillna(-1)
    df_ = df_.merge(lv_user_features, on = 'session', how='left').fillna(-1)
    print('Done 1/3')
    df_ = df_.merge(covis_f, on = ['session', 'aid'], how='left').fillna(-1)
    print('Done 2/3')


    for (type_, file_path_, index) in tqdm((zip(['carts', 'clicks', 'orders'], file_list_pops, [1,0,2])), desc = f'{iter_numb}_types'):

        folder_path = f'{main_path}/lv/{ver_folder}/test_candidates_{type_}'
        os.makedirs(folder_path, exist_ok=True)

        # Добавление признаков популярности
        columns_ = ['aid', 'rank_pops']
        pop_features = pd.read_parquet(file_path_)
        df_m = df_.merge(pop_features[columns_], on = 'aid', how='left').fillna(-1)
        print('Done 3/3')

        df_m['rank_pops'] = df_m['rank_pops'].astype(np.int32)
        df_m['rank'] = df_m['rank'].astype(np.int32)

        df_m.to_parquet(f"{main_path}/lv/{ver_folder}/test_candidates_{type_}/test_candidates_{type_}_{iter_numb}.parquet")

parts: 0it [00:00, ?it/s]

Done 1/3
Done 2/3



0_types: 0it [00:00, ?it/s]

Done 3/3



0_types: 1it [00:07,  7.89s/it]

...


4_types: 1it [00:07,  7.77s/it]

Done 3/3



4_types: 2it [00:15,  7.99s/it]

Done 3/3



4_types: 3it [00:23,  7.96s/it]
parts: 5it [03:36, 43.28s/it]


## Обучение

In [ ]:
ver = 4
ver_folder = f'ver_{ver}'
# ver_prev = 'ver_1'
typee = 'clicks'
# typee = 'carts'
# typee = 'orders'

random_state = 42

slice_value = 1

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_folder}/tr_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
tr_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    tr_candidates = pd.concat([tr_candidates, df_], ignore_index=True)

100%|██████████| 5/5 [00:44<00:00,  8.99s/it]


In [ ]:
del df_

In [ ]:
tr_candidates.session = tr_candidates.session.astype(np.int32)
tr_candidates.aid = tr_candidates.aid.astype(np.int32)

In [ ]:
# Проверка на группы
len(tr_candidates) / 150 == tr_candidates.session.nunique()

True

In [ ]:
#orders
tr_candidates['target'].value_counts()

0    77144955
1      234095
Name: target, dtype: int64

In [ ]:
#carts
tr_candidates['target'].value_counts()

0    77076368
1      302682
Name: target, dtype: int64

In [ ]:
#clicks
tr_candidates['target'].value_counts()

0    75316137
1     2062913
Name: target, dtype: int64

**Соотношение классов**

In [ ]:
#Клики
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.0015717068111910301

In [ ]:
#Картс
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.01162931726249746

In [ ]:
#Ордерс
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.001091618986545002

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# clicks
under_sampler = RandomUnderSampler(sampling_strategy=0.02)  # Указываем желаемое соотношение классов

# carts
# under_sampler = RandomUnderSampler(sampling_strategy= 0.05)  # Указываем желаемое соотношение классов

# # orders
# under_sampler = RandomUnderSampler(sampling_strategy= 0.02)  # Указываем желаемое соотношение классов


In [ ]:
FEATURES = tr_candidates.columns[: -1]
X = tr_candidates.loc[:, FEATURES]
y = tr_candidates.loc[:, 'target']
X_train, y_train = under_sampler.fit_resample(X, y)
tr_candidates = pd.merge(X_train, y_train, left_index=True, right_index=True)

In [ ]:
folder_path = 'model'
os.makedirs(folder_path, exist_ok=True)

tr_candidates = tr_candidates.sort_values('session')
tr_candidates = tr_candidates.reset_index(drop = True)

skf = GroupKFold(n_splits=5)
FEATURES = tr_candidates.columns[: -1]

for fold,(train_idx, valid_idx) in tqdm(enumerate(skf.split(tr_candidates, tr_candidates['target'], groups=tr_candidates['session'] ))):

    X_train = tr_candidates.loc[train_idx, FEATURES]
    y_train = tr_candidates.loc[train_idx, 'target']
    X_valid = tr_candidates.loc[valid_idx, FEATURES]
    y_valid = tr_candidates.loc[valid_idx, 'target']

    group_sizes_tr = X_train.groupby('session')[['aid']].count()
    group_sizes_v = X_valid.groupby('session')[['aid']].count()

    X_train = X_train.iloc[:, 2:]
    X_valid = X_valid.iloc[:, 2:]

    dtrain = xgb.DMatrix(X_train, y_train, group=group_sizes_tr.aid.tolist() )
    dvalid = xgb.DMatrix(X_valid, y_valid, group=group_sizes_v.aid.tolist() )

    xgb_parms = {'objective':'rank:pairwise',
                 'tree_method':'gpu_hist'
                #  'learning_rate' : 0.6,
                #  'max_depth' : 4
                 }
    model = xgb.train(xgb_parms,
        dtrain=dtrain,
        evals=[(dtrain,'train'),(dvalid,'valid')],
        num_boost_round=1000,
        verbose_eval=100)
    model.save_model(f'model/XGB_fold{fold}_{typee}.xgb')

0it [00:00, ?it/s]

[0]	train-map:0.91705	valid-map:0.91721
[100]	train-map:0.92311	valid-map:0.92218
[200]	train-map:0.92470	valid-map:0.92273
[300]	train-map:0.92575	valid-map:0.92284
[400]	train-map:0.92662	valid-map:0.92298
[500]	train-map:0.92741	valid-map:0.92301
[600]	train-map:0.92816	valid-map:0.92302
[700]	train-map:0.92882	valid-map:0.92293
[800]	train-map:0.92940	valid-map:0.92293
[900]	train-map:0.93003	valid-map:0.92287
[999]	train-map:0.93055	valid-map:0.92278


1it [00:52, 52.50s/it]

[0]	train-map:0.91674	valid-map:0.91724
[100]	train-map:0.92313	valid-map:0.92249
[200]	train-map:0.92471	valid-map:0.92312
[300]	train-map:0.92572	valid-map:0.92328
[400]	train-map:0.92658	valid-map:0.92337
[500]	train-map:0.92736	valid-map:0.92337
[600]	train-map:0.92806	valid-map:0.92341
[700]	train-map:0.92870	valid-map:0.92342
[800]	train-map:0.92932	valid-map:0.92337
[900]	train-map:0.92989	valid-map:0.92326
[999]	train-map:0.93044	valid-map:0.92329


2it [01:38, 48.43s/it]

...

3it [02:23, 47.15s/it]

[0]	train-map:0.91676	valid-map:0.91604
[100]	train-map:0.92317	valid-map:0.92180
[200]	train-map:0.92477	valid-map:0.92251
[300]	train-map:0.92591	valid-map:0.92264
[400]	train-map:0.92684	valid-map:0.92268
[500]	train-map:0.92760	valid-map:0.92274
[600]	train-map:0.92834	valid-map:0.92273
[700]	train-map:0.92898	valid-map:0.92277
[800]	train-map:0.92958	valid-map:0.92272
[900]	train-map:0.93016	valid-map:0.92277
[999]	train-map:0.93073	valid-map:0.92270


4it [03:09, 46.63s/it]

[0]	train-map:0.91683	valid-map:0.91742
[100]	train-map:0.92301	valid-map:0.92249
[200]	train-map:0.92457	valid-map:0.92308
[300]	train-map:0.92567	valid-map:0.92302
[400]	train-map:0.92663	valid-map:0.92311
[500]	train-map:0.92737	valid-map:0.92319
[600]	train-map:0.92806	valid-map:0.92318
[700]	train-map:0.92868	valid-map:0.92311
[800]	train-map:0.92928	valid-map:0.92300
[900]	train-map:0.92979	valid-map:0.92301
[999]	train-map:0.93033	valid-map:0.92304


5it [03:54, 46.98s/it]


In [ ]:
folder_path = f'{main_path}/models/{ver_folder}/'
os.makedirs(folder_path, exist_ok=True)

file_list = glob.glob(f'/content/model/*{typee}.xgb')
[shutil.copy(file, f'{main_path}/models/{ver_folder}/') for file in file_list]

['/content/drive/Othercomputers/Mac/Pet_project/models/ver_4/XGB_fold2_clicks.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_4/XGB_fold1_clicks.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_4/XGB_fold0_clicks.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_4/XGB_fold4_clicks.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_4/XGB_fold3_clicks.xgb']

## Предсказание

In [ ]:
model_version = 4
ver_folder = f'ver_{model_version}'

# typee = 'clicks'
# typee = 'carts'
typee = 'orders'

In [ ]:
# импорт модели
shutil.copytree(f'{main_path}/models/{ver_folder}/', f'/content/{ver_folder}/')

'/content/ver_4/'

In [ ]:
# импорт кандидатов выбранного типа
shutil.copytree(f'{main_path}/lv/{ver_folder}/test_candidates_{typee}/', f'/content/test_candidates_{typee}/' )

'/content/lv_candidates_orders/'

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'test_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
test_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    test_candidates = pd.concat([test_candidates, df_], ignore_index=True)

test_candidates = test_candidates.sort_values('session')
test_candidates = test_candidates.reset_index(drop = True)

100%|██████████| 5/5 [00:09<00:00,  1.89s/it]


In [ ]:
FEATURES = test_candidates.columns[2:]

In [ ]:
test_candidates.shape

(90062400, 10)

In [ ]:
preds = np.zeros(len(test_candidates))
for fold in tqdm(range(5)):
    model = xgb.Booster()
    model.load_model(f'{ver_folder}/XGB_fold{fold}_{typee}.xgb')
    model.set_param({'predictor': 'gpu_predictor'})
    dtest = xgb.DMatrix(data=test_candidates[FEATURES])
    preds += model.predict(dtest)/5

100%|██████████| 5/5 [01:30<00:00, 18.14s/it]


In [ ]:
lv_predictions = test_candidates[['session','aid']].copy()
lv_predictions['pred'] = preds

In [ ]:
lv_predictions = lv_predictions.sort_values(['session','pred'], ascending=[True,False]).reset_index(drop=True)
lv_predictions['n'] = lv_predictions.groupby('session').aid.cumcount().astype('int8')
lv_predictions = lv_predictions.loc[lv_predictions.n<20]

In [ ]:
lv_predictions = lv_predictions.groupby('session').aid.apply(list)
lv_predictions = lv_predictions.to_frame().reset_index()

In [ ]:
lv_predictions.rename(columns = {'aid' : 'labels'}, inplace = True)

In [ ]:
lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')

In [ ]:
shutil.copy2(f'lv_predictions_{typee}.parquet', f'{main_path}/lv/{ver_folder}/lv_predictions_{typee}.parquet' )

'/content/drive/Othercomputers/Mac/Pet_project/lv/ver_4/lv_predictions_orders.parquet'

## Оценка метрики

In [ ]:
id2type_name = 'id2type.pkl'
version = 'ver_4'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')


In [ ]:
with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
     id2type= pickle.load(file)

In [ ]:
# # Список файлов Parquet
file_list = glob.glob(f'{main_path}/lv/{version}/lv_predictions*.parquet')

# Создание пустого DataFrame для сбора данных
lv_predictions= pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df = pd.read_parquet(file)

    if 'carts' in file:
      df['type'] = 'carts'
    elif 'clicks' in file:
      df['type'] = 'clicks'
    else:
      df['type'] = 'orders'

    # Объединение временного DataFrame с основным датасетом
    lv_predictions = pd.concat([lv_predictions, df], ignore_index=True)

# lv_predictions.type = lv_predictions.type.map(lambda idx: id2type[idx])

100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  1.82it/s]


In [ ]:
print('Model score :', metric_eval(lv_predictions, valid, id2type))

Model score : (0.2962283122978089, type
carts     0.220645
clicks    0.263035
orders    0.339552
dtype: float64)


👁️**Model score : 0.296228**👁️

# Ver_4.1

- b_tr/lv_covis_matrix base
- ver_4.1 xgb
- 50 кандидатов
- фичи :
(база из 6 основных) + популярности товара по типам (1 признак ), counts из covis(featuress dict)
- 1/3 data

In [ ]:
ver = '4.1'
ver_folder = f'ver_{ver}'
cand_n = 50

In [ ]:
folder_path = f'{main_path}/tr/{ver_folder}'
os.makedirs(folder_path, exist_ok=True)

## train_candidates

### создание общего списка tr_candidates

In [ ]:
tr_candidates_sessions = pd.read_parquet(f'{main_path}/s/tr_candidates_sessions.parquet')
tr_2_first_w = pd.read_parquet(f'{main_path}/s/tr_2_first_w.parquet')

In [ ]:
covis_matrix = read_covis_to_dict(cov_ver = 'base',
                                  cov_type = 'b_tr',
                                  n = 20)

100%|███████████████████████████████████████████| 20/20 [00:15<00:00,  1.29it/s]


In [ ]:
top_pop_aids = pd.concat([tr_candidates_sessions, tr_2_first_w])
top_pop_aids = set(top_pop_aids['aid'].value_counts().index[:cand_n].tolist())

In [ ]:
%%time
# Преобразование датафрейма в словарь
tgl_dict = tr_candidates_sessions.sort_values(["session", "ts"]).groupby('session')['aid'].apply(list).to_dict()

CPU times: user 30.5 s, sys: 1.98 s, total: 32.5 s
Wall time: 32.7 s


In [ ]:
del tr_candidates_sessions, tr_2_first_w

In [ ]:
len(tgl_dict)//3

1547581

In [ ]:
# Если нужен слайс
slicee_ = list(tgl_dict.items())[:1547581] # 1547581 - одна третья данных
slicee = dict(slicee_)
tgl_dict = slicee

In [ ]:
labels = create_labels(n = cand_n,
                      inp_dict = tgl_dict,
                      covis = covis_matrix,
                      top_pop_aids = top_pop_aids,
                      cov_f = True)

100%|██████████████████████████████| 1547581/1547581 [02:30<00:00, 10316.93it/s]


Словарь с признаком-рангом по ковиз-матрице сохранен в : /Users/stanislavkrupnov/Jupyter_notebook/Pet_project/tr/ver_4.1/featuress.pickle


In [ ]:
sessions = []
aid_lists = []

for session, aids in tqdm(labels.items()):
    sessions.append(session)
    aid_lists.append(aids)

100%|████████████████████████████| 1547581/1547581 [00:00<00:00, 7469448.20it/s]


In [ ]:
tr_candidates_ = pd.DataFrame({'session': sessions, 'labels': aid_lists})

In [ ]:
tr_candidates_['session'] = tr_candidates_['session'].astype(np.int32)

In [ ]:
folder_path = f'{main_path}/tr/{ver_folder}/tr_candidates'
os.makedirs(folder_path, exist_ok=True)

partt = len(tr_candidates_)//5

for batch in tqdm(range(5), desc = 'part'):

    start = partt * batch
    end = partt + start
    print(start, end)

    if batch == 4:
        batch_ = tr_candidates_.loc[start:end]
    else:
        batch_ = tr_candidates_.loc[start:end-1]

    batch_ = batch_.explode('labels')

    batch_['labels'] = batch_['labels'].astype(np.int32)

    print(batch_[batch_['session'] == 21404]['labels'].count())

    batch_.to_parquet(f'{folder_path}/tr_candidates_{batch}.parquet')


part:   0%|                                               | 0/5 [00:00<?, ?it/s]

0 309516
50


part:  20%|███████▊                               | 1/5 [00:01<00:07,  1.80s/it]

309516 619032
0


part:  40%|███████████████▌                       | 2/5 [00:03<00:05,  1.71s/it]

...

part:  60%|███████████████████████▍               | 3/5 [00:05<00:03,  1.68s/it]

928548 1238064
0


part:  80%|███████████████████████████████▏       | 4/5 [00:06<00:01,  1.66s/it]

1238064 1547580
0


part: 100%|███████████████████████████████████████| 5/5 [00:08<00:00,  1.68s/it]


### Преобразование covis_features

In [ ]:
with open(f'{main_path}/tr/{ver_folder}/featuress.pickle', 'rb') as file:

    featuress = pickle.load(file)

features_df = (pd.DataFrame.from_dict([(session_id, aid, rank)
                                for session_id, items in featuress.items()
                                for aid, rank in items.items()]))

In [ ]:
features_df.columns = ['session','aid','rank']
features_df['session'] = features_df['session'].astype(np.int32)
features_df['aid'] = features_df['aid'].astype(np.int32)
features_df['rank'] = features_df['rank'].astype(np.uint8)

In [ ]:
folder_path = f'{main_path}/tr/{ver_folder}/covis_features'
os.makedirs(folder_path, exist_ok=True)

features_df.reset_index(drop=True, inplace=True)

partt = len(features_df)//5

for batch in tqdm(range(5), desc = 'part'):

    start = partt * batch
    end = partt * (batch + 1)
    batch_ = features_df.loc[start:end]

    batch_.to_parquet(f'{main_path}/tr/{ver_folder}/covis_features/covis_features_{batch}.parquet')


part: 100%|███████████████████████████████████████| 5/5 [00:01<00:00,  2.70it/s]


### Создание признаков  популярности товара по типам

In [ ]:
tr_candidates_sessions = pd.read_parquet(f'{main_path}/s/tr_candidates_sessions.parquet')
tr_2_first_w = pd.read_parquet(f'{main_path}/s/tr_2_first_w.parquet')
feature_name = 'pop_features'

In [ ]:
for_features_df = pd.concat([tr_candidates_sessions, tr_2_first_w])
for_features_df = for_features_df.drop(columns = ['ts'])

In [ ]:
# Сгруппируем исходный DataFrame по столбцу 'type'
grouped = for_features_df.groupby('type')

id_2_type = {0:'clicks', 1: 'carts', 2:'orders'}

# Пройдемся по каждой группе и выполним ранжирование aid по встречаемости
for name, group in tqdm(grouped):
    # Считаем количество вхождений каждого aid в текущей группе и создадим временный DataFrame
    aid_counts = group['aid'].value_counts().reset_index()
    aid_counts.columns = ['aid', 'count']

    # Добавим столбец с рангом, используя функцию rank()
    aid_counts['rank_pops'] = aid_counts['count'].rank(ascending=False, method='dense')
    aid_counts.drop(columns = ['count'], inplace = True)

    # Присоединим результат к исходной группе и добавим в общий результатный DataFrame
    group = group.drop(columns = ['session', 'type']).drop_duplicates(subset=['aid'])
    merged_group = pd.merge(group, aid_counts, on='aid', how='left')
    merged_group.fillna(-1, inplace = True)
    merged_group['rank_pops'] = merged_group['rank_pops'].astype(np.int32)
    merged_group.reset_index(inplace = True, drop = True)

    merged_group.to_parquet(f'{main_path}/tr/{ver_folder}/pop_features_{id_2_type[name]}.parquet')

100%|█████████████████████████████████████████████| 3/3 [00:05<00:00,  1.96s/it]


### Добавление признаков, таргетов к tr_candidates

In [ ]:
tr_item_features = pd.read_parquet(f'{main_path}/tr/{ver_folder}/tr_item_features.parquet')
tr_user_features = pd.read_parquet(f'{main_path}/tr/{ver_folder}/tr_user_features.parquet')

In [ ]:
targets = pd.read_parquet(f'{main_path}/s/targets.parquet')

**Добавление признаков + таргетов**

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_folder}/tr_candidates/*')
file_list_covis_f = glob.glob(f'{main_path}/tr/{ver_folder}/covis_features/*')
file_list_pops = glob.glob(f'{main_path}/tr/{ver_folder}/pop_features_*.parquet')

file_list.sort()
file_list_covis_f.sort()
file_list_pops.sort()

# Цикл для чтения и объединения файлов
for iter_numb, (file, file_2) in tqdm(enumerate(zip(file_list, file_list_covis_f)), desc = f'parts'):

    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)
    covis_f = pd.read_parquet(file_2)

    df_ = df_.rename(columns = {'labels': 'aid'})
    df_ = df_.merge(tr_item_features, on = 'aid', how='left').fillna(-1)
    df_ = df_.merge(tr_user_features, on = 'session', how='left').fillna(-1)
    print('Done 1/3')
    df_ = df_.merge(covis_f, on = ['session', 'aid'], how='left').fillna(-1)
    print('Done 2/3')


    for (type_, file_path_, index) in tqdm((zip(['carts', 'clicks', 'orders'], file_list_pops, [1,0,2])), desc = f'{iter_numb}_types'):

        folder_path = f'{main_path}/tr/{ver_folder}/tr_candidates_{type_}'
        os.makedirs(folder_path, exist_ok=True)

        # Добавление признаков популярности
        columns_ = ['aid', 'rank_pops']
        pop_features = pd.read_parquet(file_path_)
        df_m = df_.merge(pop_features[columns_], on = 'aid', how='left').fillna(-1)
        print('Done 3/3')

        df_m = add_targets(dataframe_ = df_m,
                           targets = targets,
                           targ_type = index)

        df_m['rank_pops'] = df_m['rank_pops'].astype(np.int32)
        df_m['rank'] = df_m['rank'].astype(np.int32)

        print('Done 4/3')
#         df_.to_parquet(f"{main_path}/tr/{ver_folder}/tr_candidates_{type_}/tr_candidates_{type_}_{iter_numb}.parquet")
        df_m.to_parquet(f"{main_path}/tr/{ver_folder}/tr_candidates_{type_}/tr_candidates_{type_}_{iter_numb}.parquet")

parts: 0it [00:00, ?it/s]

Done 1/3
Done 2/3



0_types: 0it [00:00, ?it/s]

Done 3/3
Done 4/3



0_types: 1it [00:06,  6.53s/it]

...


4_types: 1it [00:07,  7.81s/it]

Done 3/3
Done 4/3



4_types: 2it [00:21, 11.46s/it]

Done 3/3
Done 4/3



4_types: 3it [00:29,  9.86s/it]
parts: 5it [03:03, 36.67s/it]


In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_folder}/tr_candidates_carts/*')

# Создание пустого DataFrame для сбора данных
df = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    df = pd.concat([df, df_], ignore_index=True)

100%|█████████████████████████████████████████████| 5/5 [00:04<00:00,  1.14it/s]


In [ ]:
# Проверка на группы
len(df) / 50 == df.session.nunique()

True

In [ ]:
x = df.groupby('session')['aid'].count().reset_index()

In [ ]:
xx = df.groupby('session')['aid'].nunique().reset_index()

In [ ]:
a = 0
for session in tqdm(x['session']) :

    if (x.loc[x['session'] == session, 'aid'] != xx.loc[xx['session'] == session, 'aid']).any():
        print(session)
        a+=1

## test_candidates

### Создание списка test_candidates CPU ONLY

In [ ]:
ver = 4.1
ver_folder = f'ver_{ver}'
cand_n = 50

In [ ]:
test_candidates_sessions = pd.read_parquet(f'{main_path}/s/test_candidates_sessions.parquet')
tr_3_first_w = pd.read_parquet(f'{main_path}/s/tr_3_first_w.parquet')

In [ ]:
covis_matrix = read_covis_to_dict(cov_ver = 'base',
                                  cov_type = 'lv',
                                  n = 80)

100%|██████████| 80/80 [01:47<00:00,  1.35s/it]


In [ ]:
top_pop_aids = pd.concat([tr_3_first_w, test_candidates_sessions])
top_pop_aids = set(top_pop_aids['aid'].value_counts().index[:cand_n].tolist())

In [ ]:
# Преобразование датафрейма в словарь
test_dict = test_candidates_sessions.sort_values(["session", "ts"]).groupby('session')['aid'].apply(list).to_dict()

In [ ]:
sizee = len(test_dict) // 3

In [ ]:
# Если нужен слайс
slicee_ = list(test_dict.items())[:sizee] # sizee - одна третья данных
slicee = dict(slicee_)
test_dict = slicee

In [ ]:
labels = create_labels(n = cand_n,
                      inp_dict = test_dict,
                      covis = covis_matrix,
                      top_pop_aids = top_pop_aids,
                      cov_f = True)

100%|██████████| 600417/600417 [01:49<00:00, 5475.62it/s]


Словарь с признаком-рангом по ковиз-матрице сохранен в : /content/drive/Othercomputers/Mac/Pet_project/lv/ver_4.1/featuress.pickle


In [ ]:
sessions = []
aid_lists = []

for session, aids in tqdm(labels.items()):
    sessions.append(session)
    aid_lists.append(aids)

100%|██████████| 600417/600417 [00:00<00:00, 2148625.95it/s]


In [ ]:
test_candidates_ = pd.DataFrame({'session': sessions, 'labels': aid_lists})
test_candidates_['session'] = test_candidates_['session'].astype(np.int32)

In [ ]:
folder_path = f'{main_path}/lv/{ver_folder}/test_candidates'
os.makedirs(folder_path, exist_ok=True)

partt = len(test_candidates_)//5

for batch in tqdm(range(5), desc = 'part'):

    start = partt * batch
    end = partt + start
    print(start, end)

    if batch == 4:
        batch_ = test_candidates_.loc[start:end]
    else:
        batch_ = test_candidates_.loc[start:end-1]

    batch_ = batch_.explode('labels')

    batch_['labels'] = batch_['labels'].astype(np.int32)

    batch_.to_parquet(f'{folder_path}/test_candidates_{batch}.parquet')


part:   0%|          | 0/5 [00:00<?, ?it/s]

0 120083


part:  20%|██        | 1/5 [00:01<00:06,  1.64s/it]

120083 240166


part:  40%|████      | 2/5 [00:03<00:04,  1.64s/it]

...

part:  60%|██████    | 3/5 [00:04<00:03,  1.62s/it]

360249 480332


part:  80%|████████  | 4/5 [00:06<00:01,  1.63s/it]

480332 600415


part: 100%|██████████| 5/5 [00:08<00:00,  1.66s/it]


In [ ]:
del test_candidates_

### Преобразование covis - features

In [ ]:
with open(f'{main_path}/lv/{ver_folder}/featuress.pickle', 'rb') as file:

    featuress = pickle.load(file)

features_df = (pd.DataFrame.from_dict([(session_id, aid, rank)
                                for session_id, items in featuress.items()
                                for aid, rank in items.items()]))

In [ ]:
features_df.columns = ['session','aid','rank']
features_df['session'] = features_df['session'].astype(np.int32)
features_df['aid'] = features_df['aid'].astype(np.int32)
features_df['rank'] = features_df['rank'].astype(np.uint8)

In [ ]:
folder_path = f'{main_path}/lv/{ver_folder}/covis_features'
os.makedirs(folder_path, exist_ok=True)

features_df.reset_index(drop=True, inplace=True)

partt = len(features_df)//5

for batch in tqdm(range(5), desc = 'part'):

    start = partt * batch
    end = partt * (batch + 1)
    batch_ = features_df.loc[start:end]

    batch_.to_parquet(f'{main_path}/lv/{ver_folder}/covis_features/covis_features_{batch}.parquet')


part: 100%|██████████| 5/5 [00:01<00:00,  2.63it/s]


### Создание признаков  популярности товара по типам

In [ ]:
test_candidates_sessions = pd.read_parquet(f'{main_path}/s/test_candidates_sessions.parquet')
tr_3_first_w = pd.read_parquet(f'{main_path}/s/tr_3_first_w.parquet')
feature_name = 'pop_features'

In [ ]:
for_features_df = pd.concat([test_candidates_sessions, tr_3_first_w])
for_features_df = for_features_df.drop(columns = ['ts'])

In [ ]:
# Сгруппируем исходный DataFrame по столбцу 'type'
grouped = for_features_df.groupby('type')

id_2_type = {0:'clicks', 1: 'carts', 2:'orders'}

# Пройдемся по каждой группе и выполним ранжирование aid по встречаемости
for name, group in tqdm(grouped):
    # Считаем количество вхождений каждого aid в текущей группе и создадим временный DataFrame
    aid_counts = group['aid'].value_counts().reset_index()
    aid_counts.columns = ['aid', 'count']

    # Добавим столбец с рангом, используя функцию rank()
    aid_counts['rank_pops'] = aid_counts['count'].rank(ascending=False, method='dense')
    aid_counts.drop(columns = ['count'], inplace = True)

    # Присоединим результат к исходной группе и добавим в общий результатный DataFrame
    group = group.drop(columns = ['session', 'type']).drop_duplicates(subset=['aid'])
    merged_group = pd.merge(group, aid_counts, on='aid', how='left')
    merged_group.fillna(-1, inplace = True)
    merged_group['rank_pops'] = merged_group['rank_pops'].astype(np.int32)
    merged_group.reset_index(inplace = True, drop = True)

    merged_group.to_parquet(f'{main_path}/lv/{ver_folder}/pop_features_{id_2_type[name]}.parquet')

100%|██████████| 3/3 [00:28<00:00,  9.34s/it]


### Добавление признаков к test_candidates

In [ ]:
lv_item_features = pd.read_parquet(f'{main_path}/lv/{ver_folder}/lv_item_features.parquet')
lv_user_features = pd.read_parquet(f'{main_path}/lv/{ver_folder}/lv_user_features.parquet')

**Добавление признаков**

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/lv/{ver_folder}/test_candidates/*')
file_list_covis_f = glob.glob(f'{main_path}/lv/{ver_folder}/covis_features/*')
file_list_pops = glob.glob(f'{main_path}/lv/{ver_folder}/pop_features_*.parquet')

file_list.sort()
file_list_covis_f.sort()
file_list_pops.sort()

# Цикл для чтения и объединения файлов
for iter_numb, (file, file_2) in tqdm(enumerate(zip(file_list, file_list_covis_f)), desc = f'parts'):

    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)
    covis_f = pd.read_parquet(file_2)

    df_ = df_.rename(columns = {'labels': 'aid'})
    df_ = df_.merge(lv_item_features, on = 'aid', how='left').fillna(-1)
    df_ = df_.merge(lv_user_features, on = 'session', how='left').fillna(-1)
    print('Done 1/3')
    df_ = df_.merge(covis_f, on = ['session', 'aid'], how='left').fillna(-1)
    print('Done 2/3')


    for (type_, file_path_, index) in tqdm((zip(['carts', 'clicks', 'orders'], file_list_pops, [1,0,2])), desc = f'{iter_numb}_types'):

        folder_path = f'{main_path}/lv/{ver_folder}/test_candidates_{type_}'
        os.makedirs(folder_path, exist_ok=True)

        # Добавление признаков популярности
        columns_ = ['aid', 'rank_pops']
        pop_features = pd.read_parquet(file_path_)
        df_m = df_.merge(pop_features[columns_], on = 'aid', how='left').fillna(-1)
        print('Done 3/3')

        df_m['rank_pops'] = df_m['rank_pops'].astype(np.int32)
        df_m['rank'] = df_m['rank'].astype(np.int32)

        df_m.to_parquet(f"{main_path}/lv/{ver_folder}/test_candidates_{type_}/test_candidates_{type_}_{iter_numb}.parquet")

parts: 0it [00:00, ?it/s]

Done 1/3
Done 2/3



0_types: 0it [00:00, ?it/s]

Done 3/3



0_types: 1it [00:05,  5.96s/it]

...


4_types: 1it [00:04,  4.92s/it]

Done 3/3



4_types: 2it [00:09,  4.82s/it]

Done 3/3



4_types: 3it [00:14,  4.90s/it]
parts: 5it [02:24, 28.90s/it]


## Обучение

In [ ]:
ver = 4.1
ver_folder = f'ver_{ver}'
# ver_prev = 'ver_1'
# typee = 'clicks'
# typee = 'carts'
typee = 'orders'

random_state = 42

slice_value = 1

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_folder}/tr_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
tr_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    tr_candidates = pd.concat([tr_candidates, df_], ignore_index=True)

100%|██████████| 5/5 [00:08<00:00,  1.71s/it]


In [ ]:
del df_

In [ ]:
tr_candidates.session = tr_candidates.session.astype(np.int32)
tr_candidates.aid = tr_candidates.aid.astype(np.int32)

In [ ]:
# Проверка на группы
len(tr_candidates) / 50 == tr_candidates.session.nunique()

True

**Соотношение классов**

In [ ]:
#Клики
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.027390053209978095

In [ ]:
#Картс
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.0039270402570084776

In [ ]:
#Ордерс
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.0030344822937546597

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# clicks
# under_sampler = RandomUnderSampler(sampling_strategy=0.43)  # Указываем желаемое соотношение классов

# carts
# under_sampler = RandomUnderSampler(sampling_strategy= 0.0624)  # Указываем желаемое соотношение классов

# orders
under_sampler = RandomUnderSampler(sampling_strategy= 0.0624)  # Указываем желаемое соотношение классов


In [ ]:
FEATURES = tr_candidates.columns[: -1]
X = tr_candidates.loc[:, FEATURES]
y = tr_candidates.loc[:, 'target']
X_train, y_train = under_sampler.fit_resample(X, y)
tr_candidates = pd.merge(X_train, y_train, left_index=True, right_index=True)

In [ ]:
folder_path = 'model'
os.makedirs(folder_path, exist_ok=True)

tr_candidates = tr_candidates.sort_values('session')
tr_candidates = tr_candidates.reset_index(drop = True)

skf = GroupKFold(n_splits=5)
FEATURES = tr_candidates.columns[: -1]

for fold,(train_idx, valid_idx) in tqdm(enumerate(skf.split(tr_candidates, tr_candidates['target'], groups=tr_candidates['session'] ))):

    X_train = tr_candidates.loc[train_idx, FEATURES]
    y_train = tr_candidates.loc[train_idx, 'target']
    X_valid = tr_candidates.loc[valid_idx, FEATURES]
    y_valid = tr_candidates.loc[valid_idx, 'target']

    group_sizes_tr = X_train.groupby('session')[['aid']].count()
    group_sizes_v = X_valid.groupby('session')[['aid']].count()

    X_train = X_train.iloc[:, 2:]
    X_valid = X_valid.iloc[:, 2:]

    dtrain = xgb.DMatrix(X_train, y_train, group=group_sizes_tr.aid.tolist() )
    dvalid = xgb.DMatrix(X_valid, y_valid, group=group_sizes_v.aid.tolist() )

    xgb_parms = {'objective':'rank:pairwise',
                 'tree_method':'gpu_hist'
                #  'learning_rate' : 0.6,
                #  'max_depth' : 4
                 }
    model = xgb.train(xgb_parms,
        dtrain=dtrain,
        evals=[(dtrain,'train'),(dvalid,'valid')],
        num_boost_round=1000,
        verbose_eval=100)
    model.save_model(f'model/XGB_fold{fold}_{typee}.xgb')

0it [00:00, ?it/s]

[0]	train-map:0.97081	valid-map:0.97044
[100]	train-map:0.97634	valid-map:0.97547
[200]	train-map:0.97732	valid-map:0.97573
[300]	train-map:0.97819	valid-map:0.97579
[400]	train-map:0.97887	valid-map:0.97584
[500]	train-map:0.97953	valid-map:0.97583
[600]	train-map:0.98015	valid-map:0.97584
[700]	train-map:0.98068	valid-map:0.97584
[800]	train-map:0.98119	valid-map:0.97585
[900]	train-map:0.98169	valid-map:0.97578
[999]	train-map:0.98213	valid-map:0.97574


1it [00:29, 29.22s/it]

[0]	train-map:0.97066	valid-map:0.97041
[100]	train-map:0.97633	valid-map:0.97533
[200]	train-map:0.97734	valid-map:0.97566
[300]	train-map:0.97824	valid-map:0.97568
[400]	train-map:0.97892	valid-map:0.97572
[500]	train-map:0.97958	valid-map:0.97580
[600]	train-map:0.98012	valid-map:0.97576
[700]	train-map:0.98065	valid-map:0.97569
[800]	train-map:0.98116	valid-map:0.97565
[900]	train-map:0.98167	valid-map:0.97565
[999]	train-map:0.98210	valid-map:0.97558


2it [00:54, 26.74s/it]

...

3it [01:19, 25.95s/it]

[0]	train-map:0.97058	valid-map:0.97077
[100]	train-map:0.97626	valid-map:0.97556
[200]	train-map:0.97723	valid-map:0.97570
[300]	train-map:0.97808	valid-map:0.97584
[400]	train-map:0.97875	valid-map:0.97591
[500]	train-map:0.97939	valid-map:0.97597
[600]	train-map:0.97997	valid-map:0.97597
[700]	train-map:0.98050	valid-map:0.97601
[800]	train-map:0.98103	valid-map:0.97600
[900]	train-map:0.98151	valid-map:0.97598
[999]	train-map:0.98199	valid-map:0.97594


4it [01:44, 25.59s/it]

[0]	train-map:0.97064	valid-map:0.97026
[100]	train-map:0.97637	valid-map:0.97527
[200]	train-map:0.97736	valid-map:0.97547
[300]	train-map:0.97820	valid-map:0.97559
[400]	train-map:0.97888	valid-map:0.97554
[500]	train-map:0.97951	valid-map:0.97548
[600]	train-map:0.98012	valid-map:0.97549
[700]	train-map:0.98063	valid-map:0.97548
[800]	train-map:0.98115	valid-map:0.97552
[900]	train-map:0.98161	valid-map:0.97546
[999]	train-map:0.98203	valid-map:0.97540


5it [02:09, 25.86s/it]


In [ ]:
folder_path = f'{main_path}/models/{ver_folder}/'
os.makedirs(folder_path, exist_ok=True)

file_list = glob.glob(f'/content/model/*{typee}.xgb')
[shutil.copy(file, f'{main_path}/models/{ver_folder}/') for file in file_list]

['/content/drive/Othercomputers/Mac/Pet_project/models/ver_4.1/XGB_fold4_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_4.1/XGB_fold3_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_4.1/XGB_fold0_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_4.1/XGB_fold1_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_4.1/XGB_fold2_orders.xgb']

## Предсказание

In [ ]:
model_version = 4.1
ver_folder = f'ver_{model_version}'

# typee = 'clicks'
# typee = 'carts'
typee = 'orders'

In [ ]:
# импорт модели
shutil.copytree(f'{main_path}/models/{ver_folder}/', f'/content/{ver_folder}/')

'/content/ver_4.1/'

In [ ]:
# импорт кандидатов выбранного типа
shutil.copytree(f'{main_path}/lv/{ver_folder}/test_candidates_{typee}/', f'/content/test_candidates_{typee}/' )

'/content/lv_candidates_orders/'

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'test_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
test_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    test_candidates = pd.concat([test_candidates, df_], ignore_index=True)

test_candidates = test_candidates.sort_values('session')
test_candidates = test_candidates.reset_index(drop = True)

100%|██████████| 5/5 [00:02<00:00,  1.73it/s]


In [ ]:
FEATURES = test_candidates.columns[2:]

In [ ]:
test_candidates.shape

(30020800, 10)

In [ ]:
preds = np.zeros(len(test_candidates))
for fold in tqdm(range(5)):
    model = xgb.Booster()
    model.load_model(f'{ver_folder}/XGB_fold{fold}_{typee}.xgb')
    model.set_param({'predictor': 'gpu_predictor'})
    dtest = xgb.DMatrix(data=test_candidates[FEATURES])
    preds += model.predict(dtest)/5

100%|██████████| 5/5 [00:29<00:00,  5.96s/it]


In [ ]:
lv_predictions = test_candidates[['session','aid']].copy()
lv_predictions['pred'] = preds

In [ ]:
lv_predictions = lv_predictions.sort_values(['session','pred'], ascending=[True,False]).reset_index(drop=True)
lv_predictions['n'] = lv_predictions.groupby('session').aid.cumcount().astype('int8')
lv_predictions = lv_predictions.loc[lv_predictions.n<20]

In [ ]:
lv_predictions = lv_predictions.groupby('session').aid.apply(list)
lv_predictions = lv_predictions.to_frame().reset_index()

In [ ]:
lv_predictions.rename(columns = {'aid' : 'labels'}, inplace = True)

In [ ]:
lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')

In [ ]:
shutil.copy2(f'lv_predictions_{typee}.parquet', f'{main_path}/lv/{ver_folder}/lv_predictions_{typee}.parquet' )

'/content/drive/Othercomputers/Mac/Pet_project/lv/ver_4.1/lv_predictions_orders.parquet'

## Оценка метрики

In [ ]:
id2type_name = 'id2type.pkl'
version = 'ver_4.1'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')


In [ ]:
with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
     id2type= pickle.load(file)

In [ ]:
# # Список файлов Parquet
file_list = glob.glob(f'{main_path}/lv/{version}/lv_predictions*.parquet')

# Создание пустого DataFrame для сбора данных
lv_predictions= pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df = pd.read_parquet(file)

    if 'carts' in file:
      df['type'] = 'carts'
    elif 'clicks' in file:
      df['type'] = 'clicks'
    else:
      df['type'] = 'orders'

    # Объединение временного DataFrame с основным датасетом
    lv_predictions = pd.concat([lv_predictions, df], ignore_index=True)

# lv_predictions.type = lv_predictions.type.map(lambda idx: id2type[idx])

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  3.06it/s]


In [ ]:
print('Model score :', metric_eval(lv_predictions, valid, id2type))

Model score : (0.3774284550862039, type
carts     0.281636
clicks    0.308378
orders    0.436833
dtype: float64)


👁️**Model score : 0.37742**👁️

# Ver_5

- b_tr/lv_covis_matrix base
- ver_5 xgb
- 50 кандидатов
- фичи :
в ноушен
- 1/3 data

In [ ]:
ver = '5'
ver_folder = f'ver_{ver}'
cand_n = 50

In [ ]:
folder_path = f'{main_path}/tr/{ver_folder}'
os.makedirs(folder_path, exist_ok=True)

## train_candidates

### создание общего списка tr_candidates

In [ ]:
tr_candidates_sessions = pd.read_parquet(f'{main_path}/s/tr_candidates_sessions.parquet')
tr_2_first_w = pd.read_parquet(f'{main_path}/s/tr_2_first_w.parquet')

In [ ]:
covis_matrix = read_covis_to_dict(cov_ver = 'base',
                                  cov_type = 'b_tr',
                                  n = 20)

100%|███████████████████████████████████████████| 20/20 [00:15<00:00,  1.29it/s]


In [ ]:
top_pop_aids = pd.concat([tr_candidates_sessions, tr_2_first_w])
top_pop_aids = set(top_pop_aids['aid'].value_counts().index[:cand_n].tolist())

In [ ]:
%%time
# Преобразование датафрейма в словарь
tgl_dict = tr_candidates_sessions.sort_values(["session", "ts"]).groupby('session')['aid'].apply(list).to_dict()

CPU times: user 30.5 s, sys: 1.98 s, total: 32.5 s
Wall time: 32.7 s


In [ ]:
del tr_candidates_sessions, tr_2_first_w

In [ ]:
len(tgl_dict)//3

1547581

In [ ]:
# Если нужен слайс
slicee_ = list(tgl_dict.items())[:1547581] # 1547581 - одна третья данных
slicee = dict(slicee_)
tgl_dict = slicee

In [ ]:
labels = create_labels(n = cand_n,
                      inp_dict = tgl_dict,
                      covis = covis_matrix,
                      top_pop_aids = top_pop_aids,
                      cov_f = True)

100%|██████████████████████████████| 1547581/1547581 [02:30<00:00, 10316.93it/s]


Словарь с признаком-рангом по ковиз-матрице сохранен в : /Users/stanislavkrupnov/Jupyter_notebook/Pet_project/tr/ver_4.1/featuress.pickle


In [ ]:
sessions = []
aid_lists = []

for session, aids in tqdm(labels.items()):
    sessions.append(session)
    aid_lists.append(aids)

100%|████████████████████████████| 1547581/1547581 [00:00<00:00, 7469448.20it/s]


In [ ]:
tr_candidates_ = pd.DataFrame({'session': sessions, 'labels': aid_lists})

In [ ]:
tr_candidates_['session'] = tr_candidates_['session'].astype(np.int32)

In [ ]:
folder_path = f'{main_path}/tr/{ver_folder}/tr_candidates'
os.makedirs(folder_path, exist_ok=True)

partt = len(tr_candidates_)//5

for batch in tqdm(range(5), desc = 'part'):

    start = partt * batch
    end = partt + start
    print(start, end)

    if batch == 4:
        batch_ = tr_candidates_.loc[start:end]
    else:
        batch_ = tr_candidates_.loc[start:end-1]

    batch_ = batch_.explode('labels')

    batch_['labels'] = batch_['labels'].astype(np.int32)

    print(batch_[batch_['session'] == 21404]['labels'].count())

    batch_.to_parquet(f'{folder_path}/tr_candidates_{batch}.parquet')


part:   0%|                                               | 0/5 [00:00<?, ?it/s]

0 309516
50


part:  20%|███████▊                               | 1/5 [00:01<00:07,  1.80s/it]

309516 619032
0


part:  40%|███████████████▌                       | 2/5 [00:03<00:05,  1.71s/it]

...

part:  60%|███████████████████████▍               | 3/5 [00:05<00:03,  1.68s/it]

928548 1238064
0


part:  80%|███████████████████████████████▏       | 4/5 [00:06<00:01,  1.66s/it]

1238064 1547580
0


part: 100%|███████████████████████████████████████| 5/5 [00:08<00:00,  1.68s/it]


### Преобразование covis_features

In [ ]:
with open(f'{main_path}/tr/{ver_folder}/featuress.pickle', 'rb') as file:

    featuress = pickle.load(file)

features_df = (pd.DataFrame.from_dict([(session_id, aid, rank)
                                for session_id, items in featuress.items()
                                for aid, rank in items.items()]))

In [ ]:
features_df.columns = ['session','aid','rank']
features_df['session'] = features_df['session'].astype(np.int32)
features_df['aid'] = features_df['aid'].astype(np.int32)
features_df['rank'] = features_df['rank'].astype(np.uint8)

In [ ]:
folder_path = f'{main_path}/tr/{ver_folder}/covis_features'
os.makedirs(folder_path, exist_ok=True)

features_df.reset_index(drop=True, inplace=True)

partt = len(features_df)//5

for batch in tqdm(range(5), desc = 'part'):

    start = partt * batch
    end = partt * (batch + 1)
    batch_ = features_df.loc[start:end]

    batch_.to_parquet(f'{main_path}/tr/{ver_folder}/covis_features/covis_features_{batch}.parquet')


part: 100%|███████████████████████████████████████| 5/5 [00:01<00:00,  2.70it/s]


### Создание признаков  популярности товара по типам

In [ ]:
tr_candidates_sessions = pd.read_parquet(f'{main_path}/s/tr_candidates_sessions.parquet')
tr_2_first_w = pd.read_parquet(f'{main_path}/s/tr_2_first_w.parquet')
feature_name = 'pop_features'

In [ ]:
for_features_df = pd.concat([tr_candidates_sessions, tr_2_first_w])
for_features_df = for_features_df.drop(columns = ['ts'])

In [ ]:
# Сгруппируем исходный DataFrame по столбцу 'type'
grouped = for_features_df.groupby('type')

id_2_type = {0:'clicks', 1: 'carts', 2:'orders'}

# Пройдемся по каждой группе и выполним ранжирование aid по встречаемости
for name, group in tqdm(grouped):
    # Считаем количество вхождений каждого aid в текущей группе и создадим временный DataFrame
    aid_counts = group['aid'].value_counts().reset_index()
    aid_counts.columns = ['aid', 'count']

    # Добавим столбец с рангом, используя функцию rank()
    aid_counts['rank_pops'] = aid_counts['count'].rank(ascending=False, method='dense')
    aid_counts.drop(columns = ['count'], inplace = True)

    # Присоединим результат к исходной группе и добавим в общий результатный DataFrame
    group = group.drop(columns = ['session', 'type']).drop_duplicates(subset=['aid'])
    merged_group = pd.merge(group, aid_counts, on='aid', how='left')
    merged_group.fillna(-1, inplace = True)
    merged_group['rank_pops'] = merged_group['rank_pops'].astype(np.int32)
    merged_group.reset_index(inplace = True, drop = True)

    merged_group.to_parquet(f'{main_path}/tr/{ver_folder}/pop_features_{id_2_type[name]}.parquet')

100%|█████████████████████████████████████████████| 3/3 [00:05<00:00,  1.96s/it]


### Добавление признаков, таргетов к tr_candidates

In [ ]:
tr_item_features = pd.read_parquet(f'{main_path}/tr/ver_4.1/tr_item_features.parquet')
tr_user_features = pd.read_parquet(f'{main_path}/tr/ver_4.1/tr_user_features.parquet')
cos_sim_1_3_test = pd.read_parquet(f'{main_path}/tr/{ver_folder}/cos_sim_1_3_test.parquet')
dop_user_features = pd.read_parquet(f'{main_path}/tr/{ver_folder}/dop_user_features.parquet')
user_item_features = pd.read_parquet(f'{main_path}/tr/{ver_folder}/user_item_features.parquet')

In [ ]:
targets = pd.read_parquet(f'{main_path}/s/targets.parquet')

**Добавление признаков + таргетов**

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/ver_4.1/tr_candidates/*')
file_list_covis_f = glob.glob(f'{main_path}/tr/ver_4.1/covis_features/*')
file_list_pops = glob.glob(f'{main_path}/tr/ver_4.1/pop_features_*.parquet')

file_list.sort()
file_list_covis_f.sort()
file_list_pops.sort()

# Цикл для чтения и объединения файлов
for iter_numb, (file, file_2) in tqdm(enumerate(zip(file_list, file_list_covis_f)), desc = f'parts'):

    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)
    covis_f = pd.read_parquet(file_2)

    df_ = df_.rename(columns = {'labels': 'aid'})
    df_ = df_.merge(tr_item_features, on = ['aid'], how='left').fillna(-1)
    df_ = df_.merge(tr_user_features, on = ['session'], how='left').fillna(-1)
    df_ = df_.merge(cos_sim_1_3_test, on = ['session','aid'], how='left').fillna(-1)
    df_ = df_.merge(dop_user_features[['session','session_len']], on = ['session'], how='left').fillna(-1)
    print('Done 1/3')
    df_ = df_.merge(covis_f, on = ['session', 'aid'], how='left').fillna(-1)
    print('Done 2/3')


    for (type_, file_path_, index) in tqdm((zip(['carts', 'clicks', 'orders'], file_list_pops, [1,0,2])), desc = f'{iter_numb}_types'):

        folder_path = f'{main_path}/tr/{ver_folder}/tr_candidates_{type_}'
        os.makedirs(folder_path, exist_ok=True)

        # Добавление признаков популярности
        columns_ = ['aid', 'rank_pops']
        pop_features = pd.read_parquet(file_path_)
        df_m = df_.merge(pop_features[columns_], on = ['aid'], how='left').fillna(-1)
        print('Done 3/5')

        #Добавление last_aid_type
        df_m = df_m.merge(dop_user_features[['session', f'last_aid_type_{index}']], on = ['session'], how='left').fillna(-1)
        print('Done 4/5')
        #Добавление признаков user_item
        columns_user_item = ['session', f'freq_per_{type_}',
                             f'ratio_ses_{type_}',
                             f'type_counts_{type_}']
        df_m = df_m.merge(user_item_features[columns_user_item], on = ['session'], how='left').fillna(-1)
        print('Done 5/5')

        df_m = add_targets(dataframe_ = df_m,
                           targets = targets,
                           targ_type = index)

        df_m['rank_pops'] = df_m['rank_pops'].astype(np.int32)
        df_m['rank'] = df_m['rank'].astype(np.int32)

        print('Done 6/5')
#         df_.to_parquet(f"{main_path}/tr/{ver_folder}/tr_candidates_{type_}/tr_candidates_{type_}_{iter_numb}.parquet")
        df_m.to_parquet(f"{main_path}/tr/{ver_folder}/tr_candidates_{type_}/tr_candidates_{type_}_{iter_numb}.parquet")

parts: 0it [00:00, ?it/s]

Done 1/3
Done 2/3



0_types: 0it [00:00, ?it/s]

Done 3/5
Done 4/5
Done 5/5
Done 6/5



0_types: 1it [00:14, 14.06s/it]

...


4_types: 1it [00:17, 17.85s/it]

Done 3/5
Done 4/5
Done 5/5
Done 6/5



4_types: 2it [00:40, 20.77s/it]

Done 3/5
Done 4/5
Done 5/5
Done 6/5



4_types: 3it [00:57, 19.05s/it]
parts: 5it [06:12, 74.44s/it]


In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_folder}/tr_candidates_clicks/*')

# Создание пустого DataFrame для сбора данных
df = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    df = pd.concat([df, df_], ignore_index=True)

100%|█████████████████████████████████████████████| 5/5 [00:09<00:00,  1.97s/it]


In [ ]:
df.session.value_counts()

336369     403
5383       352
3998652    351
3043807    345
320626     331
          ... 
82533       50
3085682     50
82527       50
3085690     50
3932894     50
Name: session, Length: 1547581, dtype: int64

In [ ]:
# Проверка на группы
len(df) / 50 == df.session.nunique()

False

In [ ]:
x = df.groupby('session')['aid'].count().reset_index()

In [ ]:
xx = df.groupby('session')['aid'].nunique().reset_index()

In [ ]:
a = 0
for session in tqdm(x['session']) :

    if (x.loc[x['session'] == session, 'aid'] != xx.loc[xx['session'] == session, 'aid']).any():
        print(session)
        a+=1

## test_candidates

### Создание списка test_candidates CPU ONLY

In [ ]:
ver = 5
ver_folder = f'ver_{ver}'
cand_n = 50

In [ ]:
test_candidates_sessions = pd.read_parquet(f'{main_path}/s/test_candidates_sessions.parquet')
tr_3_first_w = pd.read_parquet(f'{main_path}/s/tr_3_first_w.parquet')

In [ ]:
covis_matrix = read_covis_to_dict(cov_ver = 'base',
                                  cov_type = 'lv',
                                  n = 80)

In [ ]:
top_pop_aids = pd.concat([tr_3_first_w, test_candidates_sessions])
top_pop_aids = set(top_pop_aids['aid'].value_counts().index[:cand_n].tolist())

In [ ]:
# Преобразование датафрейма в словарь
test_dict = test_candidates_sessions.sort_values(["session", "ts"]).groupby('session')['aid'].apply(list).to_dict()

In [ ]:
sizee = len(test_dict) // 3

In [ ]:
# Если нужен слайс
slicee_ = list(test_dict.items())[:sizee] # sizee - одна третья данных
slicee = dict(slicee_)
test_dict = slicee

In [ ]:
labels = create_labels(n = cand_n,
                      inp_dict = test_dict,
                      covis = covis_matrix,
                      top_pop_aids = top_pop_aids,
                      cov_f = True)

100%|██████████| 600417/600417 [01:49<00:00, 5475.62it/s]


Словарь с признаком-рангом по ковиз-матрице сохранен в : /content/drive/Othercomputers/Mac/Pet_project/lv/ver_4.1/featuress.pickle


In [ ]:
sessions = []
aid_lists = []

for session, aids in tqdm(labels.items()):
    sessions.append(session)
    aid_lists.append(aids)

100%|██████████| 600417/600417 [00:00<00:00, 2148625.95it/s]


In [ ]:
test_candidates_ = pd.DataFrame({'session': sessions, 'labels': aid_lists})
test_candidates_['session'] = test_candidates_['session'].astype(np.int32)

In [ ]:
folder_path = f'{main_path}/lv/{ver_folder}/test_candidates'
os.makedirs(folder_path, exist_ok=True)

partt = len(test_candidates_)//5

for batch in tqdm(range(5), desc = 'part'):

    start = partt * batch
    end = partt + start
    print(start, end)

    if batch == 4:
        batch_ = test_candidates_.loc[start:end]
    else:
        batch_ = test_candidates_.loc[start:end-1]

    batch_ = batch_.explode('labels')

    batch_['labels'] = batch_['labels'].astype(np.int32)

    batch_.to_parquet(f'{folder_path}/test_candidates_{batch}.parquet')


part:   0%|          | 0/5 [00:00<?, ?it/s]

0 120083


part:  20%|██        | 1/5 [00:01<00:06,  1.64s/it]

120083 240166


part:  40%|████      | 2/5 [00:03<00:04,  1.64s/it]

...

part:  60%|██████    | 3/5 [00:04<00:03,  1.62s/it]

360249 480332


part:  80%|████████  | 4/5 [00:06<00:01,  1.63s/it]

480332 600415


part: 100%|██████████| 5/5 [00:08<00:00,  1.66s/it]


In [ ]:
del test_candidates_

### Преобразование covis - features

In [ ]:
with open(f'{main_path}/lv/{ver_folder}/featuress.pickle', 'rb') as file:

    featuress = pickle.load(file)

features_df = (pd.DataFrame.from_dict([(session_id, aid, rank)
                                for session_id, items in featuress.items()
                                for aid, rank in items.items()]))

In [ ]:
features_df.columns = ['session','aid','rank']
features_df['session'] = features_df['session'].astype(np.int32)
features_df['aid'] = features_df['aid'].astype(np.int32)
features_df['rank'] = features_df['rank'].astype(np.uint8)

In [ ]:
folder_path = f'{main_path}/lv/{ver_folder}/covis_features'
os.makedirs(folder_path, exist_ok=True)

features_df.reset_index(drop=True, inplace=True)

partt = len(features_df)//5

for batch in tqdm(range(5), desc = 'part'):

    start = partt * batch
    end = partt * (batch + 1)
    batch_ = features_df.loc[start:end]

    batch_.to_parquet(f'{main_path}/lv/{ver_folder}/covis_features/covis_features_{batch}.parquet')


part: 100%|██████████| 5/5 [00:01<00:00,  2.63it/s]


### Создание признаков  популярности товара по типам

In [ ]:
test_candidates_sessions = pd.read_parquet(f'{main_path}/s/test_candidates_sessions.parquet')
tr_3_first_w = pd.read_parquet(f'{main_path}/s/tr_3_first_w.parquet')
feature_name = 'pop_features'

In [ ]:
for_features_df = pd.concat([test_candidates_sessions, tr_3_first_w])
for_features_df = for_features_df.drop(columns = ['ts'])

In [ ]:
# Сгруппируем исходный DataFrame по столбцу 'type'
grouped = for_features_df.groupby('type')

id_2_type = {0:'clicks', 1: 'carts', 2:'orders'}

# Пройдемся по каждой группе и выполним ранжирование aid по встречаемости
for name, group in tqdm(grouped):
    # Считаем количество вхождений каждого aid в текущей группе и создадим временный DataFrame
    aid_counts = group['aid'].value_counts().reset_index()
    aid_counts.columns = ['aid', 'count']

    # Добавим столбец с рангом, используя функцию rank()
    aid_counts['rank_pops'] = aid_counts['count'].rank(ascending=False, method='dense')
    aid_counts.drop(columns = ['count'], inplace = True)

    # Присоединим результат к исходной группе и добавим в общий результатный DataFrame
    group = group.drop(columns = ['session', 'type']).drop_duplicates(subset=['aid'])
    merged_group = pd.merge(group, aid_counts, on='aid', how='left')
    merged_group.fillna(-1, inplace = True)
    merged_group['rank_pops'] = merged_group['rank_pops'].astype(np.int32)
    merged_group.reset_index(inplace = True, drop = True)

    merged_group.to_parquet(f'{main_path}/lv/{ver_folder}/pop_features_{id_2_type[name]}.parquet')

100%|██████████| 3/3 [00:28<00:00,  9.34s/it]


### Добавление признаков к test_candidates

Кандидаты и признаки кроме новых - из версии 4.1

In [ ]:
lv_item_features = pd.read_parquet(f'{main_path}/lv/ver_4.1/lv_item_features.parquet')
lv_user_features = pd.read_parquet(f'{main_path}/lv/ver_4.1/lv_user_features.parquet')
cos_sim_1_3_test = pd.read_parquet(f'{main_path}/lv/{ver_folder}/cos_sim_1_3_test.parquet')
dop_user_features = pd.read_parquet(f'{main_path}/lv/{ver_folder}/dop_user_features.parquet')
user_item_features = pd.read_parquet(f'{main_path}/lv/{ver_folder}/user_item_features.parquet')

**Добавление признаков**

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/lv/ver_4.1/test_candidates/*')
file_list_covis_f = glob.glob(f'{main_path}/lv/ver_4.1/covis_features/*')
file_list_pops = glob.glob(f'{main_path}/lv/ver_4.1/pop_features_*.parquet')

file_list.sort()
file_list_covis_f.sort()
file_list_pops.sort()

# Цикл для чтения и объединения файлов
for iter_numb, (file, file_2) in tqdm(enumerate(zip(file_list, file_list_covis_f)), desc = f'parts'):

    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)
    covis_f = pd.read_parquet(file_2)

    df_ = df_.rename(columns = {'labels': 'aid'})
    df_ = df_.merge(lv_item_features, on = 'aid', how='left').fillna(-1)
    df_ = df_.merge(lv_user_features, on = 'session', how='left').fillna(-1)
    df_ = df_.merge(cos_sim_1_3_test, on = ['session','aid'], how='left').fillna(-1)
    df_ = df_.merge(dop_user_features[['session','session_len']], on = ['session'], how='left').fillna(-1)
    print('Done 1/3')
    df_ = df_.merge(covis_f, on = ['session', 'aid'], how='left').fillna(-1)
    print('Done 2/3')


    for (type_, file_path_, index) in tqdm((zip(['carts', 'clicks', 'orders'], file_list_pops, [1,0,2])), desc = f'{iter_numb}_types'):

        folder_path = f'{main_path}/lv/{ver_folder}/test_candidates_{type_}'
        os.makedirs(folder_path, exist_ok=True)

        # Добавление признаков популярности
        columns_ = ['aid', 'rank_pops']
        pop_features = pd.read_parquet(file_path_)
        df_m = df_.merge(pop_features[columns_], on = 'aid', how='left').fillna(-1)
        print('Done 3/3')

        #Добавление last_aid_type
        df_m = df_m.merge(dop_user_features[['session', f'last_aid_type_{index}']], on = ['session'], how='left').fillna(-1)
        print('Done 4/3')
        #Добавление признаков user_item
        columns_user_item = ['session', f'freq_per_{type_}',
                             f'ratio_ses_{type_}',
                             f'type_counts_{type_}']
        df_m = df_m.merge(user_item_features[columns_user_item], on = ['session'], how='left').fillna(-1)
        print('Done 5/3')

        df_m['rank_pops'] = df_m['rank_pops'].astype(np.int32)
        df_m['rank'] = df_m['rank'].astype(np.int32)

        df_m.to_parquet(f"{main_path}/lv/{ver_folder}/test_candidates_{type_}/test_candidates_{type_}_{iter_numb}.parquet")

parts: 0it [00:00, ?it/s]

Done 1/3
Done 2/3



0_types: 0it [00:00, ?it/s]

Done 3/3
Done 4/3
Done 5/3



0_types: 1it [00:09,  9.36s/it]

...


4_types: 1it [00:08,  8.31s/it]

Done 3/3
Done 4/3
Done 5/3



4_types: 2it [00:17,  8.80s/it]

Done 3/3
Done 4/3
Done 5/3



4_types: 3it [00:25,  8.60s/it]
parts: 5it [03:51, 46.34s/it]


## Обучение

In [ ]:
ver = 5
ver_folder = f'ver_{ver}'
# ver_prev = 'ver_1'
# typee = 'clicks'
# typee = 'carts'
typee = 'orders'

random_state = 42

slice_value = 1

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_folder}/tr_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
tr_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    tr_candidates = pd.concat([tr_candidates, df_], ignore_index=True)

100%|██████████| 5/5 [00:30<00:00,  6.08s/it]


In [ ]:
del df_

In [ ]:
tr_candidates.session = tr_candidates.session.astype(np.int32)
tr_candidates.aid = tr_candidates.aid.astype(np.int32)

In [ ]:
# Проверка на группы
len(tr_candidates) / 50 == tr_candidates.session.nunique()

True

**Соотношение классов**

In [ ]:
#Клики
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.027390053209978095

In [ ]:
#Картс
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.0039270402570084776

In [ ]:
#Ордерс
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.0030344822937546597

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# clicks
# under_sampler = RandomUnderSampler(sampling_strategy=0.43)  # Указываем желаемое соотношение классов

# carts
# under_sampler = RandomUnderSampler(sampling_strategy= 0.0624)  # Указываем желаемое соотношение классов

# orders
under_sampler = RandomUnderSampler(sampling_strategy= 0.0624)  # Указываем желаемое соотношение классов


In [ ]:
FEATURES = tr_candidates.columns[: -1]
X = tr_candidates.loc[:, FEATURES]
y = tr_candidates.loc[:, 'target']
X_train, y_train = under_sampler.fit_resample(X, y)
tr_candidates = pd.merge(X_train, y_train, left_index=True, right_index=True)

In [ ]:
del X, y

In [ ]:
folder_path = 'model'
os.makedirs(folder_path, exist_ok=True)

tr_candidates = tr_candidates.sort_values('session')
tr_candidates = tr_candidates.reset_index(drop = True)

skf = GroupKFold(n_splits=5)
FEATURES = tr_candidates.columns[: -1]

for fold,(train_idx, valid_idx) in tqdm(enumerate(skf.split(tr_candidates, tr_candidates['target'], groups=tr_candidates['session'] ))):

    X_train = tr_candidates.loc[train_idx, FEATURES]
    y_train = tr_candidates.loc[train_idx, 'target']
    X_valid = tr_candidates.loc[valid_idx, FEATURES]
    y_valid = tr_candidates.loc[valid_idx, 'target']

    group_sizes_tr = X_train.groupby('session')[['aid']].count()
    group_sizes_v = X_valid.groupby('session')[['aid']].count()

    X_train = X_train.iloc[:, 2:]
    X_valid = X_valid.iloc[:, 2:]

    dtrain = xgb.DMatrix(X_train, y_train, group=group_sizes_tr.aid.tolist() )
    dvalid = xgb.DMatrix(X_valid, y_valid, group=group_sizes_v.aid.tolist() )

    xgb_parms = {'objective':'rank:pairwise',
                 'tree_method':'gpu_hist'
                #  'learning_rate' : 0.6,
                #  'max_depth' : 4
                 }
    model = xgb.train(xgb_parms,
        dtrain=dtrain,
        evals=[(dtrain,'train'),(dvalid,'valid')],
        num_boost_round=1000,
        verbose_eval=100)
    model.save_model(f'model/XGB_fold{fold}_{typee}.xgb')

0it [00:00, ?it/s]

[0]	train-map:0.98891	valid-map:0.98889
[100]	train-map:0.99056	valid-map:0.98982
[200]	train-map:0.99121	valid-map:0.98986
[300]	train-map:0.99170	valid-map:0.98986
[400]	train-map:0.99218	valid-map:0.98983
[500]	train-map:0.99260	valid-map:0.98977
[600]	train-map:0.99297	valid-map:0.98978
[700]	train-map:0.99333	valid-map:0.98973
[800]	train-map:0.99366	valid-map:0.98973
[900]	train-map:0.99399	valid-map:0.98972
[999]	train-map:0.99430	valid-map:0.98972


1it [00:28, 28.33s/it]

[0]	train-map:0.98872	valid-map:0.98877
[100]	train-map:0.99047	valid-map:0.99006
[200]	train-map:0.99110	valid-map:0.99005
[300]	train-map:0.99162	valid-map:0.99004
[400]	train-map:0.99212	valid-map:0.98996
[500]	train-map:0.99257	valid-map:0.98997
[600]	train-map:0.99299	valid-map:0.98994
[700]	train-map:0.99336	valid-map:0.98990
[800]	train-map:0.99368	valid-map:0.98987
[900]	train-map:0.99400	valid-map:0.98984
[999]	train-map:0.99429	valid-map:0.98982


2it [00:52, 25.69s/it]

...

3it [01:15, 24.81s/it]

[0]	train-map:0.98893	valid-map:0.98866
[100]	train-map:0.99057	valid-map:0.98976
[200]	train-map:0.99122	valid-map:0.98979
[300]	train-map:0.99176	valid-map:0.98972
[400]	train-map:0.99222	valid-map:0.98973
[500]	train-map:0.99265	valid-map:0.98972
[600]	train-map:0.99304	valid-map:0.98977
[700]	train-map:0.99340	valid-map:0.98971
[800]	train-map:0.99373	valid-map:0.98965
[900]	train-map:0.99407	valid-map:0.98963
[999]	train-map:0.99435	valid-map:0.98961


4it [01:39, 24.45s/it]

[0]	train-map:0.98898	valid-map:0.98872
[100]	train-map:0.99054	valid-map:0.98977
[200]	train-map:0.99117	valid-map:0.98985
[300]	train-map:0.99171	valid-map:0.98984
[400]	train-map:0.99218	valid-map:0.98985
[500]	train-map:0.99263	valid-map:0.98980
[600]	train-map:0.99303	valid-map:0.98971
[700]	train-map:0.99341	valid-map:0.98974
[800]	train-map:0.99371	valid-map:0.98973
[900]	train-map:0.99403	valid-map:0.98966
[999]	train-map:0.99433	valid-map:0.98963


5it [02:03, 24.73s/it]


In [ ]:
folder_path = f'{main_path}/models/{ver_folder}/'
os.makedirs(folder_path, exist_ok=True)

file_list = glob.glob(f'/content/model/*{typee}.xgb')
[shutil.copy(file, f'{main_path}/models/{ver_folder}/') for file in file_list]

['/content/drive/Othercomputers/Mac/Pet_project/models/ver_5/XGB_fold4_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_5/XGB_fold3_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_5/XGB_fold0_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_5/XGB_fold1_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_5/XGB_fold2_orders.xgb']

## Предсказание

In [ ]:
model_version = 5
ver_folder = f'ver_{model_version}'

# typee = 'clicks'
# typee = 'carts'
typee = 'orders'

In [ ]:
# импорт модели
shutil.copytree(f'{main_path}/models/{ver_folder}/', f'/content/{ver_folder}/')

'/content/ver_5/'

In [ ]:
# импорт кандидатов выбранного типа
shutil.copytree(f'{main_path}/lv/{ver_folder}/test_candidates_{typee}/', f'/content/test_candidates_{typee}/' )

'/content/lv_candidates_orders/'

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'test_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
test_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    test_candidates = pd.concat([test_candidates, df_], ignore_index=True)

test_candidates = test_candidates.sort_values('session')
test_candidates = test_candidates.reset_index(drop = True)

100%|██████████| 5/5 [00:05<00:00,  1.07s/it]


In [ ]:
FEATURES = test_candidates.columns[2:]

In [ ]:
test_candidates.shape

(30020800, 10)

In [ ]:
preds = np.zeros(len(test_candidates))
for fold in tqdm(range(5)):
    model = xgb.Booster()
    model.load_model(f'{ver_folder}/XGB_fold{fold}_{typee}.xgb')
    model.set_param({'predictor': 'gpu_predictor'})
    dtest = xgb.DMatrix(data=test_candidates[FEATURES])
    preds += model.predict(dtest)/5

100%|██████████| 5/5 [00:45<00:00,  9.05s/it]


In [ ]:
lv_predictions = test_candidates[['session','aid']].copy()
lv_predictions['pred'] = preds

In [ ]:
lv_predictions = lv_predictions.sort_values(['session','pred'], ascending=[True,False]).reset_index(drop=True)
lv_predictions['n'] = lv_predictions.groupby('session').aid.cumcount().astype('int8')
lv_predictions = lv_predictions.loc[lv_predictions.n<20]

In [ ]:
lv_predictions = lv_predictions.groupby('session').aid.apply(list)
lv_predictions = lv_predictions.to_frame().reset_index()

In [ ]:
lv_predictions.rename(columns = {'aid' : 'labels'}, inplace = True)

In [ ]:
lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')

In [ ]:
shutil.copy2(f'lv_predictions_{typee}.parquet', f'{main_path}/lv/{ver_folder}/lv_predictions_{typee}.parquet' )

'/content/drive/Othercomputers/Mac/Pet_project/lv/ver_5/lv_predictions_orders.parquet'

## Оценка метрики

In [ ]:
id2type_name = 'id2type.pkl'
version = 'ver_5'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')


In [ ]:
with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
     id2type= pickle.load(file)

In [ ]:
# # Список файлов Parquet
file_list = glob.glob(f'{main_path}/lv/{version}/lv_predictions*.parquet')

# Создание пустого DataFrame для сбора данных
lv_predictions= pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df = pd.read_parquet(file)

    if 'carts' in file:
      df['type'] = 'carts'
    elif 'clicks' in file:
      df['type'] = 'clicks'
    else:
      df['type'] = 'orders'

    # Объединение временного DataFrame с основным датасетом
    lv_predictions = pd.concat([lv_predictions, df], ignore_index=True)

# lv_predictions.type = lv_predictions.type.map(lambda idx: id2type[idx])

100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  2.93it/s]


In [ ]:
print('Model score :', metric_eval(lv_predictions, valid, id2type))

Model score : (0.48735355556187143, type
carts     0.327295
clicks    0.377523
orders    0.585688
dtype: float64)


👁️**Model score : 0.48735**👁️